In [2]:
import skimage
from skimage import io
import cv2
import matplotlib.pyplot as plt
import tifffile as tfl
import os
import sys
import numpy as np
import scipy.io
from skimage import img_as_float, img_as_uint
from joblib import parallel, delayed
from scipy.linalg import solve
from skimage.util import view_as_blocks
from scipy.signal import convolve2d , correlate2d, fftconvolve
import shutil
import glob
from joblib import dump, load
from skimage.morphology import disk
from skimage.filters import median, difference_of_gaussians
from skimage.registration import phase_cross_correlation as pcc
from skimage.exposure import match_histograms
from scipy.ndimage import shift
import time
import bardensr
import pandas as pd
from skimage.measure import label,regionprops_table
from skimage import color
from skimage.morphology import extrema
from skimage.util import img_as_float
from skimage.exposure import rescale_intensity
from natsort import natsorted as nsort
from skimage.transform import rescale
from skimage.segmentation import expand_labels

2025-09-30 19:11:12.535169: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-30 19:11:12.539731: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-30 19:11:12.657905: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-30 19:11:12.660839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-30 19:11:15.086878: W tensorflow/compiler/tf2t

# I might want to change all sorted to nsort
# I might also change all the basecall outputs to a folder keyed dictionary to be safe
# Right now transform_coordinates method has cell pos and hyb pos inverted to make it correct, that means I have to interchange xy in the basecalls for hyb and cellpose label saving to revert to proper function transform_coord
# Also check for any unintentional aliasing, if possible change all direct references to .copy()

- To install fiji: wget https://downloads.imagej.net/fiji/archive/20191027-2045/fiji-linux64.zip, unzip fiji-linux64.zip
- To install MIST: Navigate to FIJI.app/plugins and run wget https://sites.imagej.net/NIST-ISG-MIST/plugins/MIST_.jar-20231103172358
    - rename to MIST_.jar
- To install fftw follow https://github-wiki-see.page/m/usnistgov/MIST/wiki/Install-Guide
    - copy libfftw3.so and libfftw3f.so to your FIJI.app/lib/fftw folder
- You might need to copy some jar files from windows installation of FIJI.app/jar to same place on ubuntu

In [3]:
import imagej
import scyjava
"""
This cell is to initialize FIJI
"""

scyjava.config.add_option('-Xmx6g')

pth_fiji = r'/home/nrg/Fiji.app' 

try:
    
    ij = imagej.init(pth_fiji, add_legacy=True)
    
    print("Initialization successful!")
    print(f"ImageJ version: {ij.getVersion()}")
    
    print(f"Legacy service is active: {ij.legacy.isActive()}")

except Exception as e:
    print(f" An error occurred during initialization: {e}")

Operating in headless mode - the original ImageJ will have limited functionality.


Initialization successful!
ImageJ version: 2.16.0/1.54p
Legacy service is active: True


In [4]:
def minmax_norm(x):
    """
    Preprocessing function: 
    Image normalization by its minimum and maximum
    """
    y=np.divide((x-np.min(x,axis=None)),np.max(x,axis=None)-np.min(x,axis=None))
    return y

In [5]:
def uint16m(x):
    """
    Preprocessing function:
    MATLAB equivalent of uint16-does clipping between data bounds and then converts
    """
    y=np.uint16(np.clip(np.round(x),0,65535))
    return y

In [6]:
def get_folders(pth):
    """
    Universal function:
    Scans the processed directory to get total tiles (folders),names, positions and x and y grid points 
    """
    dr=nsort(glob.glob(os.path.join(pth,'processed','MAX*')))
    folder=[]
    xp=[]
    yp=[]
    pos=[]
    for i,j in enumerate(dr):
        folder.append(str(j).split('/')[-1])
        temp=str(j).split('/')[-1].split('_')
        pos.append(temp[1])
        xp.append(int(temp[2])+1)
        yp.append(int(temp[3])+1)
    return folder,pos,xp,yp

In [7]:
def denoise_n2v_call(pth,basedir,fname,config_file='cfg.yaml'):
    run_diff_env_scripts("denoising_n2v",config_file,pth=pth,basedir=basedir,fname=fname)

In [8]:
def back_sub_opencv_open(I,radius,pth,name,num_c,writefile):
    """
    Preprocessing function:
    Background subtraction per tile using opencv morphological opening
    """
    I=I.copy()
    I_filtered=np.zeros_like(I)
    I_rem=I[num_c:,:,:]
    I=I[0:num_c,:,:]
    k=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(radius,radius))
    for i in range(len(I)):
        bck=cv2.morphologyEx(I[i,:,:], cv2.MORPH_OPEN, kernel= k)
        I_filtered[i,:,:]=I[i,:,:]-np.expand_dims(bck,0)
    
    I_filtered[num_c:,:,:]=I_rem    
    I_filtered=uint16m(I_filtered)
    if writefile:
        tfl.imwrite(os.path.join(pth,name),I_filtered,photometric='minisblack')
    return I_filtered

In [9]:
def channel_alignment(I_filtered,fname,config_pth,pth,name,num_c,is_affine,writefile):
    """
    Preprocessing function:
    Channel alignment per tile using only translations 
    """
    I_filtered=I_filtered.copy()
    Ishifted=np.zeros_like(I_filtered)
    I_rem=I_filtered[num_c:,:,:]
    I_filtered=I_filtered[0:num_c,:,:]
    chshift=scipy.io.loadmat(os.path.join(config_pth,fname))['chshift20x']
    for i in range(chshift.shape[0]):
        if is_affine:
            tform=chshift[i] # refine this later on-ng
        else:
            tform=skimage.transform.SimilarityTransform(translation=-chshift[i,:]) # remember this takes -shifts or use tform.inverse-ng
            
        It=skimage.transform.warp(np.squeeze(I_filtered[i,:,:]),tform,preserve_range=True,output_shape=(I_filtered.shape[1],I_filtered.shape[2]))
        Ishifted[i,:,:]=np.expand_dims(It,0)
    Ishifted[num_c:,:,:]=I_rem    
    Ishifted=uint16m(Ishifted)
    if writefile:
        tfl.imwrite(os.path.join(pth,name),Ishifted,photometric='minisblack')
    return Ishifted

In [10]:
def bleedthrough_linear(Is,num_c,config_pth,fname,pth,name,writefile):
    """
    Preprocessing function:
    Bleedthrough correction per tile 
    """
    Is=Is.copy()
    Icorrected=np.zeros_like(Is)
    chprofile=scipy.io.loadmat(os.path.join(config_pth,fname))['chprofile20x']
    Ishifted2=np.float64(Is[0:num_c,:,:])
    I_rem=Is[num_c:,:,:]
    Icorrected=(np.reshape((np.linalg.solve(np.transpose(chprofile),((np.reshape(Ishifted2,(num_c,-1),order='F'))))),(num_c,Ishifted2.shape[1],Ishifted2.shape[2]),order='F'))
    Icorrected=uint16m(Icorrected)
    Icorrected=np.append(Icorrected,I_rem,axis=0)
    if writefile:
        tfl.imwrite(os.path.join(pth,name),Icorrected,photometric='minisblack')
    return Icorrected

In [11]:
def organize_files(pth='D:/HTNA24_new/',cyclename='geneseq'):
    """
    Preprocessing function:
    Copy all cycle geneseq files from experiment directory to each folder representing one tile
    """
    print('Copying files in the correct folders...')
    genefolders=sorted(glob.glob(os.path.join(pth,cyclename+"*")))
    #print(genefolders)
    for f,i in enumerate(genefolders):
        if not(os.path.isdir(f)):
            f=[]
    
    try:
        os.makedirs(os.path.join(pth,'processed'),exist_ok=True)
        #print(pthw)
    except Exception:
        print('Directory already exists\n')
    for i,f2 in enumerate(genefolders):
        print(f'Organizing folder {genefolders[i]}\n')
        files=sorted(glob.glob(os.path.join(pth,genefolders[i],'MAX*.tif')))
        for j,f1 in enumerate(files):
            name=f1.split('MAX')
            fname=name[1].split('.')
            os.makedirs(os.path.join(pth,'processed','MAX'+fname[0]),exist_ok=True)
            shutil.copy(f1,os.path.join(pth,'processed','MAX'+fname[0],cyclename+f2.partition(cyclename)[-1]+'.tif'))
            print(f'Copied {j}')
            

In [12]:
def geneseq_cycle_alignment_block_correlation(imagename,templatename,subsample_rate,do_coarse,resize_factor,block_size,num_c,numcr,pth,folder,name):
    """
    Preprocessing function:
    Given two images-two cycles of geneseq-align image (moving) to template (fixed) using windowed phase correlation based registration
    Writes aligned image, returns aligned image and transformation matrix
    """
    os.makedirs(os.path.join(pth,folder),exist_ok=True)
        
    moving=tfl.imread(os.path.join(pth,imagename),key=range(0,num_c,1))
    fixed=tfl.imread(os.path.join(pth,templatename),key=range(0,num_c,1))
    moving_sum=np.double(np.sum(moving,axis=0))
    moving_sum=np.divide(moving_sum,np.max(moving_sum,axis=None))
    fixed_sum=np.double(np.sum(fixed,axis=0))
    fixed_sum=np.divide(fixed_sum,np.max(fixed_sum,axis=None))
    sz=fixed_sum.shape
    b_x=np.floor(sz[0]/block_size)
    b_y=np.floor(sz[1]/block_size)


    if b_x*block_size!=sz[0]:
        fixed_sum=fixed_sum[0:b_x*block_size-1,:]
        moving_sum=moving_sum[0:b_x*block_size-1,:]

    if b_y*block_size!=sz[1]:
        fixed_sum=fixed_sum[:,0:b_y*block_size-1]
        moving_sum=moving_sum[:,0:b_y*block_size-1]
        
    moving_rescaled=np.uint8(skimage.transform.rescale(moving_sum,resize_factor)*255) # check if this uint8 needs to be changed as per matlab standard-ng
    fixed_rescaled=np.uint8(skimage.transform.rescale(fixed_sum,resize_factor)*255)
    moving_split=view_as_blocks(moving_rescaled, block_shape=(block_size*resize_factor,block_size*resize_factor))
    fixed_split=view_as_blocks(fixed_rescaled, block_shape=(block_size*resize_factor,block_size*resize_factor))
    fixed_split_lin=np.reshape(fixed_split,(-1,fixed_split.shape[2],fixed_split.shape[3]))
    
    fixed_split_sum=[np.sum(j,axis=None) for i,j in enumerate(fixed_split_lin)]
    idx=np.argsort(fixed_split_sum)[::-1]
    fixed_split_sum=np.reshape(fixed_split_sum,(fixed_split.shape[0],fixed_split.shape[1]))
   
    moving_split_lin=np.reshape(moving_split,(-1,moving_split.shape[2],moving_split.shape[3]))
    
    xcorr2=lambda a,b: fftconvolve(a,np.rot90(b,k=2))
    c=np.zeros((fixed_split_lin.shape[1]*2-1,fixed_split_lin.shape[2]*2-1))
    for i in range(np.int32(np.round(fixed_split_lin.shape[0]/subsample_rate))): # check for int32-ng
        if np.max(fixed_split_lin[idx[i]],axis=None)>0:
            c=c+xcorr2(np.double(fixed_split_lin[idx[i]]),np.double(moving_split_lin[idx[i]]))

    shift_yx=np.unravel_index(np.argmax(c),c.shape)

    yoffset=-np.array([(shift_yx[0]+1-fixed_split_lin.shape[1])/resize_factor])
    xoffset=-np.array([(shift_yx[1]+1-fixed_split_lin.shape[2])/resize_factor])

    idx_minxy=np.argmin(np.abs(xoffset)+np.abs(yoffset))

    tform=skimage.transform.SimilarityTransform(translation=[xoffset[idx_minxy], yoffset[idx_minxy]])

    moving_aligned=np.zeros_like(moving)
    for i in range(moving.shape[0]):
        moving_aligned[i,:,:]=np.expand_dims(skimage.transform.warp((np.squeeze(moving[i,:,:])),tform,preserve_range=True),0)#,output_shape=(moving.shape[1],moving.shape[2])),0)# check if output size specification is necessary -ng

    moving_aligned=uint16m(moving_aligned)
    moving_aligned_full=moving_aligned.copy()
    if numcr>num_c:
        moving_last_frame=tfl.imread(os.path.join(pth,imagename),key=range(num_c,numcr,1))
        if moving_last_frame.ndim==2:
            moving_last_frame=np.expand_dims(moving_last_frame,axis=0)
        moving_aligned_full=np.append(moving_aligned_full,moving_last_frame,axis=0)
        
    tfl.imwrite(os.path.join(pth,folder,name),moving_aligned_full,photometric='minisblack')
    return moving_aligned, tform

In [13]:
# have not included default zeros shiftmat on non-existent input--ng
def mmseqalignmentsinglethread_local_subsample(pth,config_pth,cyclename,num_initial_c,num_later_c,resize_factor,block_size,chshift_filename,chprofile_filename,radius,is_affine=0,num_c=4,subsample_rate=1,do_coarse=0):
    """
    Preprocessing function:
    Does background subtraction, channel alignment, bleedthrough correction and cycle alignement for sequencing images-operates on one tile
    """
    cycle_im=sorted(glob.glob(os.path.join(pth,cyclename+"*")))
    os.makedirs(os.path.join(pth,'original'),exist_ok=True)
    os.makedirs(os.path.join(pth,'aligned'),exist_ok=True)
    fixed_filename=cycle_im.copy()
    gtforms=[]

    for i in range(len(cycle_im)):

        filename=fixed_filename[i].split("/")[-1]
        fixed_filename[i]='fixed'+filename
        if i==0:
            num_cr=num_initial_c
            templatename=fixed_filename[i]
        else:
            num_cr=num_later_c

        I=tfl.imread(cycle_im[i],key=range(0,num_cr,1))

        if radius>0:
            Ibcksub=back_sub_opencv_open(I,radius,pth,filename,num_c,0) #BACKGROUND SUBTRACTION-NG
        else:
            Ibcksub=I
    
        Ibcksub_shifted=channel_alignment(Ibcksub,chshift_filename,config_pth,pth,'bck_sub'+filename,num_c,is_affine,0) # last argument is to not to write intermediate file-ng CHANNEL ALIGN-NG
        Ibcksub_shifted_btcorr=bleedthrough_linear(Ibcksub_shifted,num_c,config_pth,chprofile_filename,pth,fixed_filename[i],1) # BLEEDTHROUGH CORRECTION-NG
        _,gtforms_ind=geneseq_cycle_alignment_block_correlation(fixed_filename[i],templatename,subsample_rate,do_coarse,resize_factor,block_size,num_c,num_cr,pth,'aligned','aligned'+filename)
        gtforms.append(gtforms_ind)

    dump(gtforms,os.path.join(pth,'tforms'+cyclename+'.joblib'))
    [os.remove(f) for f in glob.glob(os.path.join(pth,'fixed*.tif'))]
    [shutil.move(f1,os.path.join(pth,'original/'+f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pth,cyclename+'*.tif'))]
    print(f'BACKGROUND SUBTRACTION, CHANNEL ALIGNEMENT, BLEEDTHROUGH CORRECTION AND CYCLE ALIGNEMENT COMPLETE FOR FOLDER {pth.split("/")[-1]}')

In [14]:
def rgbout(pthb,config_pth,folders,cyclename,shiftmat_name=None,profilemat_name=None,align=0,bleed_correct=0,bcksub=0,m=0,strength=0.6,radius=0,num_c=4,is_affine=0):
    """
    Preprocessing function:
    From the aligned images-prepares an RGB image-channels to colors--this will be stitched later 
    """
    pthl=os.path.join(pthb,'processed',folders)
    os.makedirs(os.path.join(pthl,'RGB'),exist_ok=True)
    gname='aligned'+cyclename
    aligned_im=sorted(glob.glob(os.path.join(pthl,'aligned',gname+"*.tif")))
    filename=aligned_im.copy()

    for i in range(len(aligned_im)):
        I=tfl.imread(aligned_im[i],key=range(0,4,1))
        Ifilt=np.zeros_like(I)
        I_rgb=np.zeros((3,I.shape[1],I.shape[2]),dtype=np.double)
        if bcksub:
            Ibck=back_sub_opencv_open(I,radius,aligned_im[i],'bcksub.tif',num_c,0)
        else:
            Ibck=I
        if align:
            Ishifted=channel_alignment(Ibck,shiftmat_name,config_pth,aligned_im[i],'aligned.tif',num_c,is_affine,0)
        else:
            Ishifted=I
        if bleed_correct:
            Ifin=bleedthrough_linear(Ishifted,num_c,config_pth,profilemat_name,pth,aligned_im[i],'bcksub_aligned_btcorr.tif',0)
        else:
            Ifin=I
        if m>0:
            for ch in range(Ifin.shape[0]):
                Ifilt[ch,:,:]=median(Ifin[ch,:,:],disk(m))
        else:
            Ifilt=Ifin
            
        I_rgb[0,:,:]=np.double(Ifilt[1,:,:]+Ifilt[2,:,:]+Ifilt[3,:,:])*strength
        I_rgb[1,:,:]=np.double(Ifilt[1,:,:]+Ifilt[0,:,:]+Ifilt[3,:,:])*strength
        I_rgb[2,:,:]=np.double(Ifilt[0,:,:]+Ifilt[2,:,:]+Ifilt[3,:,:])*strength

        Irgb=np.uint8(np.clip(I_rgb,0,255))
        tfl.imwrite(os.path.join(pthl,'RGB','RGB'+filename[i].split('/')[-1]),np.transpose(Irgb,axes=(1,2,0)),photometric='rgb')


In [15]:
def align_hyb_to_gene(moving_hyb,template_sum_norm,Ibcksub_shifted_btcorr):
    """
    Preprocessing function:
    Given two images from one tile--hyb and gene--aligns hyb to geneseq
    Returns aligned image and transformation matrix
    """
    moving=moving_hyb.copy()
    moving=np.squeeze(moving,axis=0)
    moving_norm=np.divide(moving,np.max(moving,axis=None))
    moving=moving_norm
    fixed=template_sum_norm.copy()
    moving=np.uint8(np.clip(moving*255,0,255))
    fixed=np.uint8(np.clip(fixed*255,0,255))

    hmatched_moving = match_histograms(moving, fixed)
    
    shift_values,_,_ = pcc(fixed, hmatched_moving, upsample_factor=100)
    
    tform_hyb=skimage.transform.SimilarityTransform(translation=(-shift_values[1],-shift_values[0]))

    Ih_aligned=np.zeros_like(Ibcksub_shifted_btcorr)
    for i in range(Ibcksub_shifted_btcorr.shape[0]):
        Ih_aligned[i,:,:]=skimage.transform.warp(np.squeeze(Ibcksub_shifted_btcorr[i,:,:]),tform_hyb,preserve_range=True,output_shape=(template_sum_norm.shape[0],template_sum_norm.shape[1]))
    
    return Ih_aligned,tform_hyb

In [16]:
def process_hyb_images(pth,config_pth,chshift_hyb_filename,chprofile_hyb_filename,cycle_name='n2vhyb',ref_cycle_name='alignedn2vgene',reg_cycle=1,dapi_shift=[],dapi_ch_num=5,num_c_hyb=6,num_c=4,is_affine=0,radius=31,ch_i=2):
    """
    Preprocessing function:
    Does background subtraction, channel alignment, bleedthrough correction and alignement to geneseq for hyb images-operates on one tile
    """
        
    os.makedirs(os.path.join(pth,'original'),exist_ok=True)
    os.makedirs(os.path.join(pth,'aligned'),exist_ok=True)
    
    cycle_im=sorted(glob.glob(os.path.join(pth,cycle_name+"*")))
    ref_cycle_im=sorted(glob.glob(os.path.join(pth,'aligned',ref_cycle_name+"*")))
    filename=cycle_im.copy()
    ch=ch_i*np.ones((1,len(cycle_im)),dtype=np.int32)
    
    template=tfl.imread(ref_cycle_im[reg_cycle],key=range(0,num_c,1))
    
    template_sum=np.double(np.sum(template,axis=0))
    template_sum_norm=np.divide(template_sum,np.max(template_sum,axis=None))
    
    
    for i in range(len(cycle_im)):
        filename[i]=filename[i].split('/')[-1]
        Ih=tfl.imread(cycle_im[i],key=range(0,num_c_hyb,1))
        if radius>0:
            Ibcksub=back_sub_opencv_open(Ih,radius,pth,'bcksub'+filename[i],num_c_hyb-1,0)
        else:
            Ibcksub=Ih
            
        Ibcksub_shifted=channel_alignment(Ibcksub,chshift_hyb_filename,config_pth,pth,'shifted'+filename[i],num_c,is_affine,0)
        Ibcksub_shifted_btcorr=bleedthrough_linear(Ibcksub_shifted,num_c,config_pth,chprofile_hyb_filename,pth,'fixed'+filename[i],1) # BLEEDTHROUGH CORRECTION-NG
        moving_hyb=Ibcksub_shifted_btcorr[ch[i],:,:]
        [Ihyb_aligned,tform_hyb]=align_hyb_to_gene(moving_hyb,template_sum_norm,Ibcksub_shifted_btcorr)
        tfl.imwrite(os.path.join(pth,'aligned','aligned'+filename[i]),Ihyb_aligned,photometric='minisblack')
        tfl.imwrite(os.path.join(pth,'aligned','alignednuclear.tif'),Ihyb_aligned,photometric='minisblack')
        dump(tform_hyb,os.path.join(pth,'tforms_hyb_to_gene.joblib'))
        [os.remove(f) for f in glob.glob(os.path.join(pth,'fixed*.tif'))]
        [shutil.move(f1,os.path.join(pth,'original/'+f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pth,cycle_name+'*.tif'))]
        print(f'BACKGROUND SUBTRACTION, CHANNEL ALIGNEMENT, BLEEDTHROUGH CORRECTION AND HYB TO GENE ALIGNEMENT COMPLETE FOR FOLDER {cycle_im[i].split("/")[-2]}')

In [17]:
def single_align_bc_to_gene(pth,bcfname='bcseq', genefname='gene',bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5):
    """
    Preprocessing function:
    Aligns bcseq0 to geneseq0, reference channels-brightfield at this point, using phase correlation based registration, writes merged file in ??? [NEED TO CHECK WHERE IS COMP FILE WRITTEN]-operates on one tile
    Writes aligned file
    """
   
    bcfiles=sorted(glob.glob(os.path.join(pth,'*'+bcfname+'*.tif')))
    genefiles=sorted(glob.glob(os.path.join(pth,'*'+genefname+'*.tif')))
    filename=bcfiles.copy()
    
    geneim=tfl.imread(genefiles[0],key=geneidx)
    bcim=tfl.imread(bcfiles[0],key=bcidx)
   
    
    moving=bcim.copy()
    moving= difference_of_gaussians(moving, 2, 40) 
    moving_norm=np.divide(moving,np.max(moving,axis=None))
    moving=moving_norm.copy()
    fixed=geneim.copy()
    fixed= difference_of_gaussians(fixed, 2, 40) 
    fixed_norm=np.divide(fixed,np.max(fixed,axis=None))
    fixed=fixed_norm.copy()
    moving=np.uint8(np.clip(moving*255,0,255))
    fixed=np.uint8(np.clip(fixed*255,0,255))
    
    hmatched_moving = match_histograms(moving, fixed)
    
    shift_values,_,_ = pcc(fixed, hmatched_moving, upsample_factor=100)
    
    tform_bc=skimage.transform.SimilarityTransform(translation=(-shift_values[1],-shift_values[0]))
    aligned=np.zeros((3,fixed.shape[0],fixed.shape[1]),dtype=np.double)
    # aligned[0,:,:]=uint16m(geneim)
    # aligned[1,:,:]=uint16m(skimage.transform.warp(bcim,tform_bc,preserve_range=True,output_shape=(fixed.shape[0],fixed.shape[1])))
    # tfl.imwrite(os.path.join(pth,'comp.tif'),aligned,photometric='minisblack')
    aligned[0,:,:]=np.double(fixed)*0.5
    aligned[1,:,:]=np.double(skimage.transform.warp(moving,tform_bc,preserve_range=True,output_shape=(fixed.shape[0],fixed.shape[1])))*0.5
     

    aligned=np.uint8(np.clip(aligned,0,255))
    tfl.imwrite(os.path.join(pth,'comp.tif'),np.transpose(aligned,axes=(1,2,0)),photometric='rgb')



    for j in range(len(bcfiles)):
        if j==0:
            ch_r=ch_bc_init
        else:
            ch_r=ch_bc
        Ibc=tfl.imread(bcfiles[j],key=range(0,ch_r,1))
        Ibc_aligned=np.zeros_like(Ibc)
        for i in range(Ibc.shape[0]):
            Ibc_aligned[i,:,:]=skimage.transform.warp(np.squeeze(Ibc[i,:,:]),tform_bc,preserve_range=True,output_shape=(fixed.shape[0],fixed.shape[1]))
        tfl.imwrite(os.path.join(pth,'reg'+filename[j].split('/')[-1]),Ibc_aligned,photometric='minisblack')
    
    return tform_bc

In [18]:
def align_bc_to_gene(pthb,bcfname='n2vbcseq', genefname='n2vgene',bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5):
    """
    Preprocessing function:
    calls single_slign_bc_to_gene to align bcseq0 to geneseq0 for all tiles
    writes transformation matrix per tile
    """
    os.makedirs(os.path.join(pthb,'processed','BCtoGeneAlignementComparison'),exist_ok=True)
    # ALSO, I AM ONLY ALIGNING RAW FILES HERE-NOT N2V-DOESN'T MATTER BECAUSE BF IS NOT CHANGED-WHAT HAPPENS WHEN WE SWITCH TO DAPI IS ANOTHER STORY
    #organize_files(pthb,cyclename='bcseq')
    folders,_,_,_=get_folders(pthb)
    for i in range(len(folders)):
        pthl=os.path.join(pthb,'processed',folders[i])
        [shutil.move(f1,os.path.join(pthl,f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pthl,'original','n2v*.tif'))]
        [os.remove(f) for f in glob.glob(os.path.join(pthl,'*reg*.tif'))]
        tform_bc=single_align_bc_to_gene(pthl,bcfname,genefname,bcidx,geneidx,ch_bc,ch_bc_init)
        dump(tform_bc,os.path.join(pthl,'tforms_bc_to_gene.joblib'))
        [shutil.move(f1,os.path.join(pthb,'processed','BCtoGeneAlignementComparison',bcfname+'2'+genefname+'_'+folders[i]+'.tif')) for f1 in glob.glob(os.path.join(pthl,'comp.tif'))]


In [19]:
def register_seq_images_subsample(pthb,config_pth,chshift_filename,chprofile_filename,cyclename,is_affine=0,num_initial_c=5, num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2, block_size=128,radius=31,local=1,strength=0.6,m=0):
    """
    Preprocessing function:
    For all the tiles-calls functions for background subtraction, bleed-through correction, channel alignement and cycles alignement-creates RGB--moves n2v files back to original subfolder within each folder for tile
    """
    folders,_,_,_=get_folders(pthb)
    for i in range(len(folders)):
        pthl=os.path.join(pthb,'processed',folders[i])
        if os.path.isdir(os.path.join(pthl,'original')):
            [shutil.move(f1,os.path.join(pthl,f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pthl,'original/','*.tif'))]
        if local==1:
            mmseqalignmentsinglethread_local_subsample(pthl,config_pth,cyclename,num_initial_c,num_later_c,resize_factor,block_size,chshift_filename,chprofile_filename,radius,is_affine=0,num_c=4,subsample_rate=1,do_coarse=0)
        else:
            print('code some other function here')
        rgbout(pthb,config_pth,folders[i],cyclename,shiftmat_name=None,profilemat_name=None,align=0,bleed_correct=0,bcksub=0,m=0,strength=strength,radius=0,num_c=num_c,is_affine=is_affine)

        [shutil.move(f1,os.path.join(pthl,'original',f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pthl,'n2v*.tif'))]

    

In [20]:
def process_bc_cycle(pthb,config_pth,basedir,chshift_filename,chprofile_filename,config_file='cfg.yaml',is_affine=0,num_initial_c=5,num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2,block_size=128,radius=31,local=1,cyclename='regn2vbc',bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5,bcfname='n2vbcseq',genefname='n2vgene',strength=0.6,m=0):
    """
    Preprocessing function:
    For bcseq--calls functions that align bcseq0 to geneseq0, preprocess bcseq--for all tiles
    """   
    t0=time.time()
    #organize_files(pthb,cyclename='bcseq')
    #denoise_n2v_call(pthb,basedir,fname='bcseq',config_file=config_file)
    #align_bc_to_gene(pthb,bcfname, genefname,bcidx,geneidx,ch_bc,ch_bc_init)
    
    register_seq_images_subsample(pthb,config_pth,chshift_filename,chprofile_filename,cyclename,is_affine,num_initial_c, num_later_c,num_c,subsample_rate,do_coarse,resize_factor, block_size,radius,local,strength,m)
    
    tn=time.time()
    
    print(f"Total time {tn-t0} seconds\n")

In [21]:
def process_geneseq_cycle(pthb,config_pth,basedir,chshift_filename,chprofile_filename,config_file='cfg.yaml',is_affine=0,num_initial_c=5,num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2,block_size=128,radius=31,local=1,cyclename='n2vgene',strength=0.6,m=0):
    """
    Preprocessing function:
    For bcseq--calls functions that align bcseq0 to geneseq0, preprocess bcseq--for all tiles
    """   
    t0=time.time()
    organize_files(pthb,cyclename='geneseq')
    denoise_n2v_call(pthb,basedir,fname='geneseq',config_file=config_file)
    register_seq_images_subsample(pthb,config_pth,chshift_filename,chprofile_filename,cyclename,is_affine,num_initial_c, num_later_c,num_c,subsample_rate,do_coarse,resize_factor, block_size,radius,local,strength,m)
    
    tn=time.time()
    
    print(f"Total time {tn-t0} seconds\n")

In [22]:
def process_hyb_cycle(pthb,config_pth,basedir,chshift_hyb_filename,chprofile_hyb_filename,config_file='cfg.yaml',cycle_name='n2vhyb',num_c=5,is_affine=0,cyclename_rgb='n2vhyb01',strength=0.6):
    t0=time.time()
    organize_files(pthb,cyclename='hyb')
    denoise_n2v_call(pthb,basedir,fname='hyb',config_file=config_file)
    folders,_,_,_=get_folders(pthb)
    for i in range(len(folders)):
        pthl=os.path.join(pthb,'processed',folders[i])
        [shutil.move(f1,os.path.join(pth,'original/'+f1.split('/')[-1])) for f1 in glob.glob(os.path.join(pth,cycle_name+'*.tif'))]
        process_hyb_images(pthl,config_pth,chshift_hyb_filename,chprofile_hyb_filename,cycle_name='n2vhyb',ref_cycle_name='alignedn2vgene',reg_cycle=1,dapi_shift=[],dapi_ch_num=5,num_c_hyb=6,num_c=4,is_affine=0,radius=31,ch_i=2)
        rgbout(pthb,config_pth,folders[i],cyclename_rgb,shiftmat_name=None,profilemat_name=None,align=0,bleed_correct=0,bcksub=0,m=0,strength=strength,radius=0,num_c=num_c,is_affine=is_affine)
    tn=time.time()
    print(f"Total time {tn-t0} seconds\n")

In [23]:
def produce_tform_and_stitched_images(pth,sx,sy,intensity_scaling=3,tilesize=3200,transform_rescale_factor=0.5,display_additional_rescale=0.2):
    """
    Stitching function:
    1. For all tiles, save the rescaled (downscaled) transformation per tile, apply transformation on RGB of each tile, stitch all the tramsformed RGB into one big image and store it in check_registration folder
    2. Calls function to create a merged transform file for global position estimation
    """ 
    [folders,pos,x,y]=get_folders(pth)
    unique_pos=nsort(np.unique(pos))
    check_reg_folder=os.path.join(pth,'processed','checkregistration')
    os.makedirs(check_reg_folder,exist_ok=True)
    folder_names=np.array(folders)
    filenames=sorted(glob.glob(os.path.join(pth,'processed',folders[0],'RGB',"*"+".tif")))
    T=load(os.path.join(pth,'tforms_rescaled'+str(transform_rescale_factor).replace('.','p')+'.joblib'))
    for f,filename in enumerate(filenames):
        filename=filename.split('/')[-1]
        I=np.zeros([int(np.max(sy)+tilesize*transform_rescale_factor),int(np.max(sx)+tilesize*transform_rescale_factor),3])
        #ax.imshow(I,cmap='jet',vmin=0,vmax=250)
        for n_pos in unique_pos:
            pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
            for ids in pos_id:
                tilename=folder_names[ids]+'.tif'
                tform=skimage.transform.SimilarityTransform(scale=transform_rescale_factor,translation=[T[n_pos][tilename]["ref_pos"][0],T[n_pos][tilename]["ref_pos"][1]]) 
                dump(tform,os.path.join(pth,'processed',folder_names[ids],'global_tform_'+str(transform_rescale_factor).replace('.','p')+'.joblib'))
                tile=tfl.imread(os.path.join(pth,'processed',folder_names[ids],'RGB',filename))
                # if is_grayscale:
                #     It=skimage.transform.warp(np.squeeze(tile[stitching_channel-1,:,:]),tform.inverse,preserve_range=True,output_shape=(I.shape[0],I.shape[1]))
                # else:
                It=skimage.transform.warp(np.squeeze(tile),tform.inverse,preserve_range=True,output_shape=(I.shape[0],I.shape[1]))
                I=np.maximum(I,It)
   
        Irgb=np.uint8(np.clip(I*intensity_scaling,0,255))
        Irgb_rescaled = np.uint8(np.clip(rescale(Irgb, display_additional_rescale, channel_axis=-1, preserve_range=True, anti_aliasing=True),0,255))
        tfl.imwrite(os.path.join(check_reg_folder,n_pos+'_'+filename),Irgb_rescaled,photometric='rgb')
    merge_transforms(pth,str(transform_rescale_factor).replace('.','p'))

In [24]:
def merge_transforms(pth,name):
    """
    Stitching function:
    Merge all downscaled transforms per tile into a final transformation file
    """ 
    [folders,_,_,_]=get_folders(pth)
    T={}
    for folder in folders:
        T[folder]=load(os.path.join(pth,'processed',folder,'global_tform_'+name+'.joblib'))
    dump(T,os.path.join(pth,'processed','tforms_final.joblib'))

In [25]:
def stitch_mist_allpos(pth,pth_fiji,fname='n2vgeneseq01',stitching_channel=3,overlap=0.23,numcores=8,is_headless=True,grid_direction='UR',px=0.33,transform_rescale_factor=0.5,num_c=4):
    #DON'T KNOW WHY STITCHING CHANNEL IS MENTIONED, MAYBE I WILL COPY ONLY REF CHANNELS LATER, NOT DONE IN THE ORIGINAL MATLAB EITHER
    """
    Stitching function:
    1. MIST based stitching--copies all geneseq0 tiles to another stitch folder from where MIST can pick up and stitch
    2. For each unique position (slice)-calls MIST for all the tiles belonging to that position (slice)
    3. Calls function to merge mist outputs
    4. Calls function to rescale the transformations
    """ 
    [folders,pos,x,y]=get_folders(pth)
    unique_pos=nsort(np.unique(pos))
    pth_input_stitch=os.path.join(pth,'processed','stitch',fname,'ref_tiles')
    pth_output_stitch=os.path.join(pth,'processed','stitch',fname,'stitched')
    os.makedirs(pth_input_stitch,exist_ok=True)
    os.makedirs(pth_output_stitch,exist_ok=True)
    for folder in folders:
        shutil.copy(os.path.join(pth,'processed',folder,'original',fname+'.tif'),os.path.join(pth,'processed','stitch',fname,'ref_tiles',folder+'.tif'))
    x=np.array(x)
    y=np.array(y)
    for n_pos in unique_pos:
        pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
        print(pos_id)
        grid_x=np.max(x[pos_id])
        grid_y=np.max(y[pos_id])
        name=folders[pos_id[0]].split('_')[0:2]
        outprefix=name[0]+'_'+name[1]+'_'
        filename=outprefix+'{ccc}_{rrr}.tif'
        stitch_mist_single_pos(grid_x,grid_y,pth_input_stitch,pth_output_stitch,filename,grid_direction,outprefix,px,numcores,pth_fiji,overlap,is_headless)
    merge_mist_results(unique_pos,pth_output_stitch,pth)
    sx,sy=rescale_transformation(pth,folders,unique_pos,pos,transform_rescale_factor,num_c)
    return sx,sy

    

In [26]:
def stitch_mist_single_pos(grid_x,grid_y,pth_input_stitch,pth_output_stitch,filename,grid_direction,outprefix,px,numcores,pth_fiji,overlap=0.23,is_headless=True):
    """
    Stitching function:
    MIST based stitching--per position--calls IJ MIST plugin
    """ 
    pth_fftwlib=os.path.join(pth_fiji,'lib','fftw')
    os.makedirs(pth_fftwlib,exist_ok=True)
    pth_fftplan=os.path.join(pth_fftwlib,'fftPlans')
    os.makedirs(pth_fftplan,exist_ok=True)
    config={
        'gridwidth':grid_x,
        'gridheight':grid_y,
        'starttilerow':0,
        'starttilecol':0,
        'imagedir':pth_input_stitch,
        'filenamepattern':filename,
        'filenamepatterntype':'ROWCOL',
        'gridorigin':grid_direction,
        'assemblefrommetadata':False,
        'assemblenooverlap':False,
        'globalpositionsfile':[],
        'numberingpattern':'HORIZONTALCOMBING',
        'startrow':0,
        'startcol':0,
        'extentwidth':grid_x,
        'extentheight':grid_y,
        'timeslices':0,
        'istimeslicesenabled':False,
        'outputpath':pth_output_stitch,
        'displaystitching':False,
        'outputfullimage':False,
        'outputmeta':True,
        'outputimgpyramid':False,
        'blendingmode':'AVERAGE',
        'blendingalpha':0.5,
        'compressionmode':'UNCOMPRESSED',
        'outfileprefix':outprefix,
        'unit':'MICROMETER',
        'unitx':px,
        'unity':px,
        'programtype':'AUTO',
        'numcputhreads':numcores,
        'loadfftwplan':True,
        'savefftwplan':True,
        'fftwplantype':'MEASURE',
        'ffftwlibraryname':'libfftw3',
        'ffftwlibraryfilename':'libfftw3.so',
        'planpath':pth_fftplan,
        'ffftwlibrarypath':pth_fftwlib,
        'stagerepeatability':0,
        'horizontaloverlap':overlap*100,
        'verticaloverlap':overlap*100,
        'numfftpeaks':10,
        'overlapuncertainty':'NaN',
        'isusedoubleprecision':True,
        'isusebioformats':False,
        'issuppressmodelwarningdialog':False,
        'isenablecudaexceptions':False,
        'translationrefinementmethod':'EXHAUSTIVE',
        'numtranslationrefinementstartpoints':32,
        'headless':is_headless,
        'loglevel':'VERBOSE',
        'debuglevel':'None'    
    }
    ij.py.run_plugin("MIST", config)

In [27]:
def merge_mist_results(unique_pos,pth_output_stitch,pth):
    """
    Stitching function:
    1. MIST based stitching results are encoded in a global dictionary 
    2. For each position (slice)-position of tiles and their names is saved as a sub-dictionary
    3. One final dictionary with positions as keys is stored as tforms_original file
    
    """ 
    Texp={}
    for n_pos in unique_pos:
        lines=[]

        Tfull={}
        with open(os.path.join(pth_output_stitch,'MAX_'+n_pos+'_global-positions-0.txt'), 'r') as file:
            while True:
                line = file.readline()
                if not line:
                    break
                lines.append(line.strip())

        for line in lines:
            T={}
            fields=line.split(';')
            for field in fields:
       
                dict_pair=(field.split(':',1))
                key=dict_pair[0].strip()
                value=dict_pair[-1].strip()
                if key=='file':
                    tilename=value
                elif key=='position':
                    T[key]=[int(pos) for pos in value.strip('()').split(',')]
                elif key=='grid':
                    T[key]=[int(grid) for grid in value.strip('()').split(',')]
        
            Tfull[tilename]=T
            T={}
    
        Texp[n_pos]=Tfull

        Tfull={}
    dump(Texp,os.path.join(pth,'processed','tforms_original.joblib'))


In [28]:
def merge_ashlar_results(pth,transform_rescale_factor=0.5,num_c=4):
    """
    Stitching function:
    1. ASHLAR based stitching results are encoded in a global dictionary 
    2. For each position (slice)-position of tiles and their names is saved as a sub-dictionary
    3. One final dictionary with positions as keys is stored as tforms_original file
    4. Calls function to rescale transformation
    """ 
    [folders,pos,_,_]=get_folders(pth)
    unique_pos=nsort(np.unique(pos))
    folder_names=np.array(folders)
    Texp={}
    Tfull={}
    for n_pos in unique_pos:
        T={}
        df=pd.read_csv(os.path.join(pth,'MAX_'+n_pos+'.positions.tsv'), sep='\t')
        pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
        for ids in pos_id:
            tilename=folder_names[ids]+'.tif'
            T['position']=[df.iloc[ids,2],df.iloc[ids,1]]
            T['grid']=[0,0]
            Tfull[tilename]=T
            T={}
        Texp[n_pos]=Tfull
        Tfull={}
    dump(Texp,os.path.join(pth,'processed','tforms_original.joblib'))
    sx,sy=rescale_transformation(pth,folders,unique_pos,pos,transform_rescale_factor,num_c)
    return sx,sy

In [29]:
def rescale_transformation(pth,folders,unique_pos,pos,rescale_factor=0.5,num_c=4):
    """
    Stitching function:
    1. Reads the original transformation dictionary
    2. Downscales the coordinates as per rescale_factor and write in as new key-value pairs per tile in the original dictionary
    3. Writes the modified dictionary
    
    """ 

    folder_names=np.array(folders)
    T=load(os.path.join(pth,'tforms_original.joblib'))
    sx=[]
    sy=[]
    for n_pos in unique_pos:
        pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
        for ids in pos_id:
            tilename=folder_names[ids]+'.tif'
            T[n_pos][tilename]["ref_pos"]=[T[n_pos][tilename]["position"][0]*rescale_factor,T[n_pos][tilename]["position"][1]*rescale_factor]
            sx.append(T[n_pos][tilename]["ref_pos"][0])
            sy.append(T[n_pos][tilename]["ref_pos"][1])
        
    #pprint.pprint(T)
    dump(T,os.path.join(pth,'processed','tforms_rescaled'+str(rescale_factor).replace('.','p')+'.joblib'))
    return sx,sy

In [30]:
def basecall_hyb_call(pth,thresh,prominence,config_pth,relaxed=1,no_deconv=1,filter_overlap=0,codebook_hyb_name='codebookhyb.mat',cyclename='alignedn2vhyb',num_c=4,all_genes_ch=2):
    """
    Basecalling function:
    Calls basecalling hyb function--LOOKS WEIRD TO ME :) WHY WOULD I CODE CALL TO A CALL--I AM TIRED :) :)
    """ 
    basecall_hyb_all_tiles(pth,cyclename,config_pth,codebook_hyb_name,thresh,prominence,num_c,all_genes_ch,no_deconv)

In [31]:
def basecall_hyb_all_tiles(pth,cyclename,config_pth,codebook_hyb_name,thresh,prominence,num_c=4,all_genes_ch=2,no_deconv=1):
    """
    Basecalling function:
    1. Basecalls hyb for all tiles
    2. Writes combined basecall results for all tiles
    """ 
    # TASK FOR LATER-----THIS PRODUCES NESTED LIST---CHANGE IT BY APPENDING ONLY [0] OF EACH NESTED LIST LIKE BC BASECALL

    [folders,_,_,_]=get_folders(pth)
    codebook_hyb_path=os.path.join(config_pth,codebook_hyb_name)
    codebook_hyb=scipy.io.loadmat(codebook_hyb_path)['codebookhyb']
    lroi_x_all=[]
    lroi_y_all=[]
    id_t_all=[]
    sig_t_all=[]
    for folder in folders:
        hybseq=sorted(glob.glob(os.path.join(pth,'processed',folder,'aligned',cyclename+"*"+"tif")))
        m=len(hybseq)

        if m==1:
            hyb_raw=tfl.imread(os.path.join(hybseq[0]),key=range(0,num_c,1))
            if no_deconv:
                hyb_2=hyb_raw
                hyb_2[all_genes_ch,:,:]=0
                [lroi_x_ind,lroi_y_ind,id_t_ind,sig_t_ind]=basecall_hyb_one_image(os.path.join(pth,'processed',folder,'aligned'),num_c,all_genes_ch,hyb_2,thresh,prominence)
            else:
                print('Deconvolution to be introduced')
        else:
            print('Multiple hybseq basecall to be developed')
        lroi_x_all.append(lroi_x_ind)
        lroi_y_all.append(lroi_y_ind)
        id_t_all.append(id_t_ind)
        sig_t_all.append(sig_t_ind)
        print(f'HYB BASECALL COMPLETE FOR FOLDER {hybseq[0].split("/")[-3]}')
    dump({"lroi_x":lroi_x_all,"lroi_y":lroi_y_all,"gene_id":id_t_all,"signal":sig_t_all},os.path.join(pth,'processed','genehyb'+'.joblib'))
    


In [32]:
from skimage.morphology import binary_dilation
from skimage.measure import regionprops
def basecall_hyb_one_image(pthw,num_c,all_genes_ch,hyb_2,thresh,prominence):
    """
    Basecalling function: I FEEL I SHOULD COMBINE MASK AND CALL ONCE-TO REMOVE OVERLAP
    1. Basecalls hyb for one tile
    2. Writes segmented rolony image per tile
    3. Returns basecall results to the calling function
    """ 
    lroi_x=[]
    lroi_y=[]
    id_t=[]
    sig_t=[]
    mask=np.zeros_like(hyb_2)
    for n in range(num_c):
        if n==all_genes_ch:
            mask[n,:,:]=0
            continue
        else:
            a=hyb_2[n,:,:].copy()
            a_mask=a>thresh[n]
            a_masked=a*a_mask
            a_max= extrema.h_maxima(a_masked,prominence[n])
            label_peaks = label(a_max)
            m=regionprops(label_peaks,a_masked)
            mask[n,:,:]=uint16m(binary_dilation(a_max))
            [lroi_x,lroi_y,id_t,sig_t]=quantify_peaks(lroi_x,lroi_y,id_t,sig_t,m,hyb_2)

    tfl.imwrite(os.path.join(pthw,'mask_hyb.tif'),mask,photometric='minisblack')
    return(lroi_x,lroi_y,id_t,sig_t)

In [33]:
def quantify_peaks(lroi_x,lroi_y,id_t,sig_t,m,hyb_2):
    """
    Basecalling function:
    1. Based on the regionprops results per tile, this function creates hyb basecalling output and decodes the gene
    2. Returns the basecall output to the calling function
    """ 
    sig1=[]
    lroi1_x=[]
    lroi1_y=[]
    id1=[]
    for i,peaks in enumerate(m):
        lroi1_x.append(peaks.centroid[0])
        lroi1_y.append(peaks.centroid[1])
        sig1.append(peaks.intensity_max)
        id1.append(np.argmax(hyb_2[:,peaks.coords[0][0],peaks.coords[0][1]])+1) # added 1 here to max channel to match codebook 1,2 and 4
    lroi_x.append(lroi1_x)
    lroi_y.append(lroi1_y)
    id_t.append(id1)
    sig_t.append(sig1)
    return(lroi_x,lroi_y,id_t,sig_t)

In [34]:
def basecall_barcodes_rolony(pth,relaxed=0,thresh=[30,30,30,30],prominence=[30,30,30,30],num_cycles=15,num_c=4):
    [folders,pos,x,y]=get_folders(pth)
    lroi_x_all=[]
    lroi_y_all=[]
    id_t_all=[]
    sig_t_all=[]
    score_t_all=[]
    seq_t_all=[]
    for folder in folders:
        print(f'BARCODE BASECALL FOLDER {folder}')
        pthw=os.path.join(pth,'processed',folder,'aligned')
        I=[]
        for i in range(num_cycles):
            I.append(tfl.imread(os.path.join(pthw,'alignedregn2vbcseq'+str("%0.2d"%(i+1))+'.tif'),key=range(0,4,1)))
        [lroi_x,lroi_y,id_t,sig_t,score_t,seq_t]=basecall_bc_one_image(pthw,num_c,I,thresh,prominence)
        lroi_x_all.append(lroi_x[0])
        lroi_y_all.append(lroi_y[0])
        id_t_all.append(id_t[0])
        sig_t_all.append(sig_t[0])
        score_t_all.append(score_t[0])
        seq_t_all.append(seq_t[0])
    score_t_all=np.array(score_t_all)
    score_t_all[np.isnan(score_t_all)]=0.5
    score_t_all=score_t_all.tolist()
    dump({"lroi_x_all":lroi_x_all,"lroi_y_all":lroi_y_all,"id_t_all":id_t_all,"sig_t_all":sig_t_all,"score_t_all":score_t_all,"seq_t_all":seq_t_all},os.path.join(pth,'processed','bc.joblib'))



In [35]:
def basecall_bc_one_image(pthw,num_c,I,thresh,prominence):
    """
    Basecalling function:
    1. Basecalls bc for one tile
    2. Writes segmented rolony image per tile
    3. Returns basecall results to the calling function
    """ 
    lroi_x=[]
    lroi_y=[]
    id_t=[]
    sig_t=[]
    score_t=[]
    mask=np.zeros_like(I[0])
    for n in range(num_c):
        a=I[0][n,:,:]
        a_mask=a>thresh[n]
        a_masked=a*a_mask
        a_max= extrema.h_maxima(a_masked,prominence[n])
        
        
        mask[n,:,:]=uint16m(binary_dilation(a_max))# OK, SO I WILL REMOVE DILATION FROM HERE AND DILATE AFTER OVERLAP REMOVAL IN NEXT LINE
    comb_mask=clear_overlapping_rolonies(np.max(mask,axis=0))
    tfl.imwrite(os.path.join(pthw,'mask_bc.tif'),mask,photometric='minisblack')
    tfl.imwrite(os.path.join(pthw,'comb_mask_bc.tif'),comb_mask,photometric='minisblack')
    label_peaks = label(comb_mask)
    m=regionprops(label_peaks,comb_mask)
    
    [lroi_x,lroi_y,id_t,sig_t,score_t,seq]=quantify_peaks_bc(lroi_x,lroi_y,id_t,sig_t,score_t,m,I)   

    dump({"lroi_x":lroi_x,"lroi_y":lroi_y,"id_t":id_t,"sig_t":sig_t,"score_t":score_t,"seq":seq},os.path.join(pthw,'basecalls-bc.joblib'))

    return(lroi_x,lroi_y,id_t,sig_t,score_t,seq)
    

In [36]:
def clear_overlapping_rolonies(a):
    c=np.array([[0,1,1],[0,0,1],[0,0,1]])
    a=a & skimage.util.invert(binary_dilation(a,c))
    return a

In [37]:
def quantify_peaks_bc(lroi_x,lroi_y,id_t,sig_t,score_t,m,I):
    """
    Basecalling function:
    1. Based on the regionprops results per tile, this function creates bc basecalling output and decodes the gene
    2. Returns the basecall output to the calling function
    """ 
    sig2=[]
    lroi1_x=[]
    lroi1_y=[]
    id2=[]
    score2=[]
    gene_map=np.array(list("GTAC")) 
    
    for i,peaks in enumerate(m):
        lroi1_x.append(peaks.centroid[0])
        lroi1_y.append(peaks.centroid[1])
        score1=[]
        sig1=[]
        id1=[]
        #score=0
        for j in range(len(I)):
            intensity=I[j][:,peaks.coords[0][0],peaks.coords[0][1]]
            #score[np.isnan(score)]=0.5
            sig1.append(np.max(intensity))
            id1.append(np.argmax(intensity))
            score1.append(np.max(intensity)/np.sqrt(np.sum(np.square(intensity))))
        sig2.append(sig1)
        id2.append(id1)
        score2.append(score1)
    lroi_x.append(lroi1_x)
    lroi_y.append(lroi1_y)
    id_t.append(id2)
    sig_t.append(sig2)
    score_t.append(score2)
    seq=gene_map[id_t]   
    return(lroi_x,lroi_y,id_t,sig_t,score_t,seq)

In [38]:
def basecall_bc_soma_all(pth,num_ch=4,mname='dil_cell_mask_cyto3.tif',fname='alignedregn2vbcseq'):
    bc_label_all=[]
    bc_sig_all_channels_all=[]
    bc_sig_all=[]
    bc_id_all=[]
    bc_score_all=[]
    bc_seq_all=[]
    [folders,pos,x,y]=get_folders(pth)

    for folder in folders:
        [bc_label,bc_sig_all_channels,bc_sig,bc_id,bc_score,bc_seq]=basecall_bc_soma_one_image(pth,folder,num_ch,mname,fname)
        bc_sig_all_channels_all.append(bc_sig_all_channels)
        bc_sig_all.append(bc_sig)
        bc_id_all.append(bc_id)
        bc_score_all.append(bc_score)
        bc_seq_all.append(bc_seq)
        bc_label_all.append(bc_label)
        print(f'Basecalled soma-bc folder {folder}')

    dump({"bc_label":bc_label_all,"bc_sig_all_channels":bc_sig_all_channels_all,"bc_sig":bc_sig_all," bc_id": bc_id_all,"bc_score":bc_score_all,"bc_seq":bc_seq_all},os.path.join(pth,'processed','all_bccells_intensity.joblib'))


In [39]:

def basecall_bc_soma_one_image(pth,folder,num_ch=4,mname='dil_cell_mask_cyto3.tif', fname='alignedregn2vbcseq'):
    # NUCLEAR PROFILE BASED BACKGROUND SUBTRACTION NOT DONE
    gene_map=np.array(list("GTAC"))   
    mask=tfl.imread(os.path.join(pth,'processed',folder,'aligned',mname))
    bc_label_cycles=[]
    bc_sig_all_channels_cycles=[]
    bc_sig_cycles=[]
    bc_id_cycles=[]
    bc_score_cycles=[]
    bc_seq_cycles=[]
    if np.max(mask[:])>0:
        for i in range(num_cycles):
            bc_label=[]
            bc_sig_all_channels=[]
            bc_sig=[]
            bc_id=[]
            bc_score=[]
            bc_seq=[]
            I=np.transpose(tfl.imread(os.path.join(pth,'processed',folder,'aligned',fname+str("%0.2d"%(i+1))+'.tif'),key=range(0,num_ch,1)), axes=(1,2,0))
            cell_data=regionprops_table(mask,I,properties=('label','intensity_mean'))
            bc_label=cell_data['label']
            bc_sig_all_channels=[cell_data['intensity_mean-0'],cell_data['intensity_mean-1'],cell_data['intensity_mean-2'],cell_data['intensity_mean-3']] # [channel][cells]
            bc_sig=np.max(np.array(bc_sig_all_channels),axis=0)
            bc_id=np.argmax(np.array(bc_sig_all_channels),axis=0)
            bc_score=bc_sig/np.sqrt(np.sum(np.square(bc_sig_all_channels),axis=0))
            bc_score[np.isnan(bc_score)]=0.5
            bc_seq=gene_map[bc_id]
        
            bc_label_cycles=bc_label
            bc_sig_all_channels_cycles.append(bc_sig_all_channels)
            bc_sig_cycles.append(bc_sig)
            bc_id_cycles.append(bc_id)
            bc_score_cycles.append(bc_score)
            bc_seq_cycles.append(bc_seq)
            
    dump({"bc_label":bc_label_cycles,"bc_sig_all_channels":bc_sig_all_channels_cycles,"bc_sig":bc_sig_cycles," bc_id": bc_id_cycles,"bc_score":bc_score_cycles,"bc_seq":bc_seq_cycles},os.path.join(pth,'processed',folder,'aligned','bc-somas.joblib'))
    
    return bc_label_cycles,bc_sig_all_channels_cycles,bc_sig_cycles,bc_id_cycles,bc_score_cycles,bc_seq_cycles


In [40]:
def make_codebook_bin(pth,num_channels=4,codebook_name='codebookM1all.mat'):
    """
    Basecalling function for geneseq:
    1. In preparation for bardensr-this function creates binarized codebook as per the number of geneseq cycles in the experiment
    """ 
    [folders,_,_,_]=get_folders(pth)
    genecycles=len(sorted(glob.glob(os.path.join(pth,'processed',folders[0],'original',"*geneseq*"))))    
    codebook=scipy.io.loadmat(os.path.join(config_pth,codebook_name))['codebook']
    genes=np.reshape(np.array([str(x[0][0]) for x in codebook]),(np.size(codebook,0),-1))
    codebook1=np.zeros((np.size(codebook,0),genecycles),dtype=str)

    for i in range(np.size(codebook,0)):
        for j in range(genecycles):
            codebook1[i,j]=codebook[i][1][0][j].copy() # CHECK IF COPY IS OK

    codebook_bin=np.ones(np.shape(codebook1),dtype=np.double)
    codebook_bin=np.reshape(np.array([float(x.replace('G','8').replace('T','4').replace('A','2').replace('C','1')) for y in codebook1 for x in y]),np.shape(codebook1))

    codebook_bin=np.matmul(np.uint8(codebook_bin),2**np.transpose(np.array((np.arange(4*genecycles-4,-1,-4)))))

    codebook_bin=np.array([bin(i)[2:].zfill(genecycles*num_channels) for i in codebook_bin])
    codebook_bin=np.reshape([np.uint8(i) for j in codebook_bin for i in j],(np.size(codebook1,0),genecycles*num_channels))

    co=[[genes[i],codebook_bin[j,:]] for i in range(np.size(genes,0))]
    co=[codebook,co]
    codebook_bin1=np.reshape(codebook_bin,(np.size(codebook_bin,0),-1,num_channels))
    dump(co,os.path.join(pth,'processed','codebook.joblib'))
    dump(codebook_bin1,os.path.join(pth,'processed','codebookforbardensr.joblib'))



In [41]:
def bardensr_call(pth,config_pth,num_channels=4,codebook_name='codebookM1all.mat',fdrthresh=0.05,trim=160,cropf=0.4,noisefloor_ini=0.01,noisefloor_final=0.05):
    """
    Basecalling function for geneseq:
    1. Calls the function that creates binarized codebook as per the number of geneseq cycles in the experiment
    2. Calculates normalization factors, intensity thresholds for spot detection based on allowed fdr
    3. Calls Bardensr spot calling and peak detection to identify and decode rolonies per tile
    4. Saves basecalling results per tile
    """ 
    make_codebook_bin(pth,num_channels,codebook_name)

    controlidx=range(25,35)
    [folders,_,_,_]=get_folders(pth)
    codebook=scipy.io.loadmat(os.path.join(config_pth,codebook_name))['codebook']
    genes=np.array([str(x[0][0]) for x in codebook],dtype=str)
    cb=load(os.path.join(pth,'processed','codebookforbardensr.joblib'))
    cb=np.transpose(cb,axes=(1,2,0))
    R,C,J=cb.shape
    codeflat=np.reshape(cb,(-1,J))
    pos_unused_codes=np.where(np.char.startswith(genes,'unused'))
    err_codes=genes[pos_unused_codes]

    # SELECTING 10 FOLDERS AS CONTROL
    if len(folders)>=36:
        control_folders=folders[controlidx]
    else:
        control_folders=folders

    # NORMALIZATION-PREPROCESSING--CALCULATING MAX OF EACH CYCLE AND EACH CHANNEL ACROSS ALL CONTROL FOVS
    max_per_RC=[image_reader_cropped(os.path.join(pth,'processed',i,'aligned'),R,C,cropf).max(axis=(1,2,3)) for i in control_folders]
    median_max=np.median(max_per_RC,axis=0)

    # ESTABLISHING BASE THRESHOLD AT THE MEDIAN OF MAXIMUM ERROR READOUT 

    err_max=[]
    evidence_tensors=[]
    for folder in control_folders:
        et=bardensr.spot_calling.estimate_density_singleshot(image_reader_trimmed(os.path.join(pth,'processed',folder,'aligned'),trim,R,C)/median_max[:,None,None,None],codeflat,noisefloor_ini)
        err_max.append(et[:,:,:,pos_unused_codes].max(axis=(0,1,2)))
    err_max=np.array(err_max)
    thresh=np.median(np.median(err_max,axis=1))
    print(thresh)

    # FINDING OPTIMUM TRHESHOLD WITH LOWEST FDR ON CONTROL FOLDERS

    err_c_all=[]
    total_c_all=[]
    for folder in control_folders:
        et=bardensr.spot_calling.estimate_density_singleshot(image_reader_cropped(os.path.join(pth,'processed',folder,'aligned'),R,C,cropf)/median_max[:,None,None,None],codeflat,noisefloor_final)
        for thresh1 in np.linspace(thresh-0.1,thresh+0.1,10):
            spots=bardensr.spot_calling.find_peaks(et,thresh1,use_tqdm_notebook=False)
            spots.to_csv(os.path.join(pth,'processed',folder,'aligned','spots.csv'),index=False)
            print(f"in {folder} found {len(spots)} spots")
            err_c=0
            for err_idx in pos_unused_codes[0]:
                err_c=err_c+(spots.j==err_idx).to_numpy().sum()
            err_c_all.append(err_c)
            total_c_all.append(len(spots)-err_c)      
    #calculate fdr        
    err_c_all1=np.reshape(err_c_all,[len(control_folders),10])
    total_c_all1=np.reshape(total_c_all,[len(control_folders),10])+1
    fdr=err_c_all1/len(pos_unused_codes[0])*(len(genes)-len(pos_unused_codes[0]))/(total_c_all1)
    fdrmean=err_c_all1.mean(axis=0)/len(pos_unused_codes[0])*(len(genes)-len(pos_unused_codes[0]))/(total_c_all1.mean(axis=0))
    thresh_refined=np.linspace(thresh-0.1,thresh+0.1,10)[(fdrmean<fdrthresh).nonzero()[0][0]]#this is the new threshold optimized by targeted fdr value
    print(thresh_refined)  
    with open(os.path.join(pth,'processed','thresh_refined.txt'),'w') as f:
        f.write(str(thresh_refined))
    with open(os.path.join(pth,'processed','noise_floors.txt'),'w') as f:
        f.write(str(noisefloor_final))
    print(f"threshold {thresh_refined} with noise floor {noisefloor_final}")

    # MAIN BASE-CALLING ON ALL FOLDERS-TRIMMED IMAGES WITH FINALIZED THRESHOLD
    # # spot call each fov, using the thresholds we decided on, and the normalization we decided on
    for folder in folders:
        et=bardensr.spot_calling.estimate_density_singleshot(image_reader_trimmed(os.path.join(pth,'processed',folder,'aligned'),trim,R,C)/median_max[:,None,None,None],codeflat,noisefloor_final)
        spots=bardensr.spot_calling.find_peaks(et,thresh_refined,use_tqdm_notebook=False)
        spots.loc[:,'m1']=spots.loc[:,'m1']+trim
        spots.loc[:,'m2']=spots.loc[:,'m2']+trim
        spots.to_csv(os.path.join(pth,'processed',folder,'aligned','bardensrresult.csv'),index=False)
        print(f"in {folder} found {len(spots)} spots")


In [42]:
def import_bardensr_results(pth,fname='bardensrresult.csv',prev_codebook_len=0,codebook_name='codebook.joblib',is_optseq=0,codebook_opt_name='codebook.mat'):
    """
    Basecalling function for geneseq:
    1. Calls an accumulator function to aggregate bardensr results per tile
    2. Calls function to compute fdr for all tiles combined
    3. Writes aggregated geneseq basecall results
    """
    lroi_x=[] 
    lroi_y=[]
    gene_id=[]

    [lroi_x,lroi_y,gene_id]=accumulate_bardensr_results(pth,codebook_name,fname,0)
    fdr=get_fdr(pth,codebook_name,gene_id)
    print(f'Finished importing bardensr results. FPR is {fdr}')

    if is_optseq:
        print('Has optseq')
        codebook_path=os.path.join(pth,'processed',codebook_opt_name)
        codebook=scipy.io.loadmat(codebook_path)['codebook']
        [lroi_x_opt,lroi_y_opt,gene_id_opt]=accumulate_bardensr_results(pth,codebook_opt_name,fname_opt,len(codebook))
        fdr_o=get_fdr(pth,codebook_opt_name,gene_id_opt)
        gene_id.append(gene_id_opt)
        lroi_x.append(lroi_x_opt)
        lroi_y.append(lroi_y_opt)
    dump({"lroi_x":lroi_y,"lroi_y":lroi_x,"gene_id":gene_id},os.path.join(pth,'processed','basecalls'+'.joblib')) # let's check if this works --exchanging x and y

In [43]:
def accumulate_bardensr_results(pth,codebook_name,fname,prev_codebook_len):
    """
    Basecalling function for geneseq:
    1. Accumulator function that combines bardensr results per tile
    2. Returns combined basecall results to the calling function
    3. WHY AM I READING CODEBOOK HERE?----------------CHECK AGAIN-----------------
    """
    # codebook_path=os.path.join(pth,'processed',codebook_name)
    # codebook=scipy.io.loadmat(codebook_path)['codebook']

    [folders,_,_,_]=get_folders(pth)
    lroi_x=[] 
    lroi_y=[]
    gene_id=[]

    for i,folder in enumerate(folders):
        try:
            T=pd.read_csv(os.path.join(pth,'processed',folder,'aligned',fname),header=0)
            lroi_x.append(T.m2)
            lroi_y.append(T.m1)
            gene_id.append(T.j+prev_codebook_len)
        except:
            lroi_x.append([])
            lroi_y.append([])
            gene_id.append([])
            print(f'No geneseq rolonies found for tile {folder}\n')

    return(lroi_x,lroi_y,gene_id)

In [44]:
def get_fdr(pth,codebook_name,gene_id):
    """
    Basecalling function for geneseq:
    Calculates fdr
    """
    codebook_path=os.path.join(pth,'processed',codebook_name)
    #codebook=scipy.io.loadmat(codebook_path)['codebook']
    codebook_full=load(codebook_path)
    codebook=codebook_full[0]
    genes=np.array([str(x[0][0]) for x in codebook],dtype=str)
    pos_unused_codes=np.where(np.char.startswith(genes,'unused'))
    err_codes=genes[pos_unused_codes]
    id_flat = np.array([code for pos in gene_id for code in pos])
    err_c_all1=id_flat[np.isin(id_flat,pos_unused_codes[0])]
    fdr=(len(err_c_all1)/len(id_flat))/len(pos_unused_codes[0])*(len(genes)-len(pos_unused_codes[0]))
    return(fdr)

In [45]:
def image_reader_cropped(pthb,R,C,cropf=0.4):
    """
    Helper function for basecalling function for geneseq:
    Returns a nearly center cropped in xy image stack (with cycles and channels intact)
    """
    I=[]
    for i in range(1,R+1):
        for j in range(C):
            I.append(np.expand_dims(tfl.imread(os.path.join(pthb,'alignedn2vgeneseq'+'%0.2d'%i+'.tif'),key=j),axis=0))
    I=np.array(I)
    nx=np.size(I,3)
    ny=np.size(I,2)
    return(I[:,:,round(ny*cropf):round(ny*(1-cropf)),round(nx*cropf):round(nx*(1-cropf))])

In [46]:
def image_reader_trimmed(pthb,trim,R,C):
    """
    Helper function for basecalling function for geneseq:
    Returns a border trimmed in xy image stack (with cycles and channels intact)
    """
    I=[]
    for i in range(1,R+1):
        for j in range(C):
            I.append(np.expand_dims(tfl.imread(os.path.join(pthb,'alignedn2vgeneseq'+'%0.2d'%i+'.tif'),key=j),axis=0))
    I=np.array(I)

    return(I[:,:,trim:-trim,trim:-trim])

In [47]:
def prepare_cellpose_input(pth,nuc_ch=5,num_chyb=5,num_cgene=4,tilesize=3200):
    """
    Cell segmentation function:
    1. creates an input image stack for cellpose input per tile with nuclear  information from DAPI (hyb) and gene from rest of the hyb and all geneseqs
    """
    [folders,pos,x,y]=get_folders(pth)
    other_ch=list(range(0,num_chyb))
    del other_ch[nuc_ch-1]
    cell_inp=np.zeros([2,tilesize,tilesize])
    for folder in folders:
        Ih=tfl.imread(os.path.join(pth,'processed',folder,'aligned','alignedn2vhyb01.tif'),key=range(0,num_chyb,1))
        Ig=np.zeros([Ih.shape[1],Ih.shape[2]])
        gene_cycles=sorted(glob.glob(os.path.join(pth,'processed',folder,'aligned','alignedfixedn2vgeneseq*.tif')))
        for gene_cycle in gene_cycles:
            Ig=Ig+np.sum(tfl.imread(gene_cycle,key=range(0,num_cgene,1)),axis=0)
        Inuc=Ih[nuc_ch-1,:,:]
        Icyto=np.sum(Ih[other_ch,:,:],axis=0)+Ig
        cell_inp[0,:,:]=uint16m(Icyto)
        cell_inp[1,:,:]=uint16m(Inuc)
        tfl.imwrite(os.path.join(pth,'processed',folder,'aligned','cell_inp2.tif'),uint16m(cell_inp),photometric='minisblack')


In [48]:
from skimage.segmentation import expand_labels
from skimage.measure import label,regionprops_table

def import_single_tile_segmentation(pth,folder,dilation_radius=3,fname='cell_mask_cyto3.tif'):
    """
    Cell segmentation function:
    1. Reads the output from cellpose per tile--the label mask image
    2. Dilates it by few pixels to increase cellular coverage
    3. Assigns properties to each segmented cell and returns these properties per tile to the calling function
    """
    mask=tfl.imread(os.path.join(pth,'processed',folder,'aligned',fname))
    mask_dil=expand_labels(mask,dilation_radius)
    measure=regionprops_table(mask_dil,properties=('label','centroid'))
    cell_num=measure['label']
    cent_x=measure['centroid-0']
    cent_y=measure['centroid-1']

    return mask,mask_dil,cell_num,cent_x,cent_y


In [49]:
def import_cellpose_all_tiles(pth,dilation_radius=3,fname='cell_mask_cyto3.tif'):
    """
    Cell segmentation function:
    1. Calls single tile fucntion to measure properties of cells--aggregates them for all tiles
    2. Writes the combined segmentation result for all tiles
    """
    [folders,_,_,_]=get_folders(pth)
    all_segmentation_dict={}
    for i,folder in enumerate(folders):
        tile_data={}
        [mask,mask_dil,cell_num,cent_x,cent_y]=import_single_tile_segmentation(pth,folder,dilation_radius,fname)
        tile_data['original_labels']=mask
        tile_data['dilated_labels']=mask_dil
        tile_data['cell_num']=cell_num
        tile_data['cent_x']=cent_x
        tile_data['cent_y']=cent_y
        all_segmentation_dict[folder]=tile_data
        io.imsave(os.path.join(pth,'processed',folder,'aligned','dil_'+fname),mask_dil)
    dump(all_segmentation_dict,os.path.join(pth,'processed','all_segmentation.joblib'))


In [50]:
def get_cellid(mask,coord_x,coord_y):
    """
    Global transformation function:
    1. For any detected rolony-assigns it to a cell
    2. Returns the cell ids for all rolonies in this tile
    """
    coord_xl=[int(np.round(x)) for x in coord_x]
    coord_yl=[int(np.round(x)) for x in coord_y]
    cell_id=mask[coord_xl,coord_yl]
    return cell_id

In [51]:
def assign_rolony_to_cell(mask,coord_x,coord_y):
    """
    Global transformation function:
    1. Calls get_cellid function if there are rolonies detected in this tile or else assigns empty cell id to this tile
    """
    if len(coord_x):
        cell_id=get_cellid(mask,coord_x,coord_y)
    else:
        cell_id=[] # earlier this was [] and was causing error later
    return cell_id

In [52]:
def aggregate_cell_ids_gene_hyb(pth,is_bc):
    """
    Global transformation function:
    1. Combines gene and hyb basecalls and assigns original per tile position and cell id to each
    2. Writes the combined rolony properties file
    """
    gene_rol=load(os.path.join(pth,'processed','basecalls.joblib'))
    seg=load(os.path.join(pth,'processed','all_segmentation.joblib'))
    hyb_rol=load(os.path.join(pth,'processed','genehyb.joblib'))
    if is_bc:
        bc_rol=load(os.path.join(pth,'processed','bc.joblib'))
    
    [folders,_,_,_]=get_folders(pth)
    T={}
    Tbc={}
    for i,folder in enumerate(folders):
        print(f'Operating on {folder}')
        t={}
        mask=seg[folder]['dilated_labels']
        coord_xg=gene_rol['lroi_x'][i]
        coord_yg=gene_rol['lroi_y'][i]
        coord_xh=hyb_rol['lroi_x'][i][0]
        coord_yh=hyb_rol['lroi_y'][i][0]
        t['cellid']=assign_rolony_to_cell(mask,coord_xg,coord_yg)
        t['cellidhyb']=assign_rolony_to_cell(mask,coord_xh,coord_yh)
        T[folder]=t
        if is_bc:
            tbc={}
            coord_xb=bc_rol['lroi_x_all'][i][0]
            coord_yb=bc_rol['lroi_y_all'][i][0]
            tbc['cellidbc']=assign_rolony_to_cell(mask,coord_xb,coord_yb)
            Tbc[folder]=tbc
    dump(T,os.path.join(pth,'processed','cell_id.joblib'))
    if is_bc:
        dump(Tbc,os.path.join(pth,'processed','bccellid.joblib'))
    print('ALL ROLONIES ASSIGNED TO CELLS')



In [53]:
def transform_downsized_coordinates(pth,is_bc):
    gene_rol=load(os.path.join(pth,'processed','basecalls.joblib'))
    seg=load(os.path.join(pth,'processed','all_segmentation.joblib'))
    hyb_rol=load(os.path.join(pth,'processed','genehyb.joblib'))
    Tf=load(os.path.join(pth,'processed','tforms_final.joblib'))
    if is_bc:
        bc_rol=load(os.path.join(pth,'processed','bc.joblib'))
    [folders,_,_,_]=get_folders(pth)
    T={}
    for i,folder in enumerate(folders):
        t={}
        tform=Tf[folder]
        [x,y]=apply_transform(tform,gene_rol['lroi_y'][i],gene_rol['lroi_x'][i])
        t['lroi10x_x']=x
        t['lroi10x_y']=y
        [x,y]=apply_transform(tform,hyb_rol['lroi_y'][i][0],hyb_rol['lroi_x'][i][0]) 
        t['lroi10xhyb_x']=x
        t['lroi10xhyb_y']=y
        [x,y]=apply_transform(tform,seg[folder]['cent_y'],seg[folder]['cent_x']) 
        t['cellpos10x_x']=x
        t['cellpos10x_y']=y
        if is_bc:
            [x,y]=apply_transform(tform,bc_rol['lroi_y_all'][i][0],bc_rol['lroi_x_all'][i][0]) 
            t['bcpos10x_x']=x
            t['bcpos10x_y']=y
        T[folder]=t
    dump(T,os.path.join(pth,'processed','lroi10x.joblib'))
    print('ALL COORDINATES TRANSFORMED')


In [54]:
def apply_transform(tform,coord_x,coord_y):
    """
    Global transformation function:
    1. Transforms the local coordinates of rolonies and cells to global downsized coordinates per tile
    """
    
    if len(coord_x):
        if not (isinstance(coord_x,list) or isinstance(coord_x,np.ndarray)):
            coord_x=coord_x.to_list()
            coord_y=coord_y.to_list()
        q=np.zeros([len(coord_x),2])
        q[:,0]=np.reshape(coord_x,(1,-1))
        q[:,1]=np.reshape(coord_y,(1,-1))
        v=tform(q)
        x=v[:,0]
        y=v[:,1]
    else:
        x=[]
        y=[]
        
    return x,y

In [55]:
from scipy.sparse import coo_matrix
import datetime


def organize_processed_data(pth,config_pth,is_optseq=0,hyb_codebook_name='codebookhyb.mat',starting_slice_idx=1,starting_fov_idx=1,dummy_cell_num=10000,tilesize=3200,
                           fraction_border=0.07):
    """
    Postprocessing function:
    Organizes all the data into rolonies and neurons--prepares a big dictionary
    """
    today=datetime.date.today().strftime('%d%m%Y')
    gene_rol=load(os.path.join(pth,'processed','basecalls.joblib'))
    seg=load(os.path.join(pth,'processed','all_segmentation.joblib'))
    hyb_rol=load(os.path.join(pth,'processed','genehyb.joblib'))
    Tf=load(os.path.join(pth,'processed','tforms_final.joblib'))
    coord=load(os.path.join(pth,'processed','lroi10x.joblib'))
    codebook=load(os.path.join(pth,'processed','codebook.joblib'))
    cellid=load(os.path.join(pth,'processed','cell_id.joblib'))
    hyb_codebook=scipy.io.loadmat(os.path.join(config_pth,hyb_codebook_name))['codebookhyb']
    dump([hyb_codebook],os.path.join(pth,'processed','hyb_codebook.joblib'))
    hyb_codebook=load(os.path.join(pth,'processed','hyb_codebook.joblib'))
    [folders,pos,_,_]=get_folders(pth)
    npos=nsort(np.unique(pos))
    d={}
    d=data_dict_organizer(d,'initialize',fov=[],gene_rol_id=[],pos_10x_allx=[],pos_10x_ally=[],pos_40x_allx=[],pos_40x_ally=[],
                          cellidall=[],sliceidall=[],hyb_rol_id=[],fov_hyb=[],pos_10x_allx_hyb=[],pos_10x_ally_hyb=[],pos_40x_allx_hyb=[],pos_40x_ally_hyb=[],
                          cellidall_hyb=[],sliceidall_hyb=[],cell_list_all=[],cell_pos_10x_allx=[],cell_pos_10x_ally=[],cell_pos_40x_allx=[],cell_pos_40x_ally=[],
                          fov_cell=[],sliceidall_cell=[])

    for i,folder in enumerate(folders):
        pos_id=np.array([j for j,name in enumerate(npos) if name==pos[i]]) # search for slice/position number for this tile

        d=data_dict_organizer(d,'append',
                              fov=np.full(len(gene_rol['gene_id'][i]),i),
                              gene_rol_id=np.array(gene_rol['gene_id'][i]),
                              pos_10x_allx=coord[folders[i]]['lroi10x_x'],
                              pos_10x_ally=coord[folders[i]]['lroi10x_y'],
                              pos_40x_allx=np.array(gene_rol['lroi_x'][i]),
                              pos_40x_ally=np.array(gene_rol['lroi_y'][i]),
                              cellidall=np.array(cellid[folders[i]]['cellid'])+np.array(i*starting_fov_idx*dummy_cell_num),# if len(cellid[folders[i]]['cellid']) else np.array([0]),
                              sliceidall=np.full(len(gene_rol['gene_id'][i]),pos_id+starting_slice_idx), # check this later,does it require -1 or not
                              hyb_rol_id=hyb_rol['gene_id'][i][0],
                              fov_hyb=np.full(len(hyb_rol['gene_id'][i][0]),i),
                              pos_10x_allx_hyb=coord[folders[i]]['lroi10xhyb_x'],
                              pos_10x_ally_hyb=coord[folders[i]]['lroi10xhyb_y'],
                              pos_40x_allx_hyb=hyb_rol['lroi_x'][i][0],
                              pos_40x_ally_hyb=hyb_rol['lroi_y'][i][0],
                              cellidall_hyb=np.array(cellid[folders[i]]['cellidhyb'])+np.array(i*starting_fov_idx*dummy_cell_num),
                              sliceidall_hyb=np.full(len(hyb_rol['gene_id'][i][0]),pos_id+starting_slice_idx),
                              cell_list_all=np.array(seg[folders[i]]['cell_num'])+np.array(i*starting_fov_idx*dummy_cell_num),
                              cell_pos_10x_allx=coord[folders[i]]['cellpos10x_x'],
                              cell_pos_10x_ally=coord[folders[i]]['cellpos10x_y'],
                              cell_pos_40x_allx=seg[folders[i]]['cent_x'],
                              cell_pos_40x_ally=seg[folders[i]]['cent_y'],
                              fov_cell=np.full(len(seg[folders[i]]['cell_num']),i),
                              sliceidall_cell=np.full(len(seg[folders[i]]['cell_num']),pos_id+starting_slice_idx))

    d=data_dict_organizer(d,'concat',fov=[],gene_rol_id=[],pos_10x_allx=[],pos_10x_ally=[],pos_40x_allx=[],pos_40x_ally=[],
                          cellidall=[],sliceidall=[],hyb_rol_id=[],fov_hyb=[],pos_10x_allx_hyb=[],pos_10x_ally_hyb=[],pos_40x_allx_hyb=[],pos_40x_ally_hyb=[],
                          cellidall_hyb=[],sliceidall_hyb=[],cell_list_all=[],cell_pos_10x_allx=[],cell_pos_10x_ally=[],cell_pos_40x_allx=[],cell_pos_40x_ally=[],
                          fov_cell=[],sliceidall_cell=[])
    if is_optseq:
        print('Has optseq')
        codebook_optseq=load(os.path.join(pth,'processed','codebook_optseq.joblib'))
        d['hyb_rol_id1']=d['hyb_rol_id']+len(codebook[0])-1+len(codebook_optseq[0])-1
        codebook_comb=[codebook[0],codebook_optseq[0],hyb_codebook[0]]
    else:
        d['hyb_rol_id1']=d['hyb_rol_id']+len(codebook[0])-1
        codebook_comb=[codebook[0],hyb_codebook[0]]
    codebook_comb=np.concatenate(codebook_comb)
    d=merge_gene_hyb_dict(d,'gene_rol_id','hyb_rol_id1','combined_gene_hyb_id')
    d=merge_gene_hyb_dict(d,'fov','fov_hyb','combined_gene_hyb_fov')
    d=merge_gene_hyb_dict(d,'pos_10x_allx','pos_10x_allx_hyb','combined_gene_hyb_pos10x_x')
    d=merge_gene_hyb_dict(d,'pos_10x_ally','pos_10x_ally_hyb','combined_gene_hyb_pos10x_y')
    d=merge_gene_hyb_dict(d,'pos_40x_allx','pos_40x_allx_hyb','combined_gene_hyb_pos40x_x')
    d=merge_gene_hyb_dict(d,'pos_40x_ally','pos_40x_ally_hyb','combined_gene_hyb_pos40x_y')
    d=merge_gene_hyb_dict(d,'cellidall','cellidall_hyb','combined_gene_hyb_cellidall')
    d=merge_gene_hyb_dict(d,'sliceidall','sliceidall_hyb','combined_gene_hyb_sliceidall')

    
    border_size=np.round(fraction_border*tilesize)

    pos_id=d['combined_gene_hyb_id']>0 # uncalled rolonies--how does this happen? what's bardensr's code for uncalled ones
    pos_inside_border_x=(d['combined_gene_hyb_pos40x_x']>border_size-1) & (d['combined_gene_hyb_pos40x_x']<tilesize-border_size+1)
    pos_inside_border_y=(d['combined_gene_hyb_pos40x_y']>border_size-1) & (d['combined_gene_hyb_pos40x_y']<tilesize-border_size+1)
    filter_id=pos_id & pos_inside_border_x & pos_inside_border_y

    filtered_d={}
    filtered_d=data_dict_organizer(filtered_d,'initialize',combined_gene_hyb_id=[],combined_gene_hyb_fov=[],combined_gene_hyb_pos10x_x=[],combined_gene_hyb_pos10x_y=[],
                                   combined_gene_hyb_pos40x_x=[],combined_gene_hyb_pos40x_y=[],combined_gene_hyb_cellidall=[],combined_gene_hyb_sliceidall=[])

    filtered_d=data_dict_organizer(filtered_d,'append',
                                   combined_gene_hyb_id=d['combined_gene_hyb_id'][filter_id],
                                   combined_gene_hyb_fov=d['combined_gene_hyb_fov'][filter_id],
                                   combined_gene_hyb_pos10x_x=d['combined_gene_hyb_pos10x_x'][filter_id],
                                   combined_gene_hyb_pos10x_y=d['combined_gene_hyb_pos10x_y'][filter_id],
                                   combined_gene_hyb_pos40x_x=d['combined_gene_hyb_pos40x_x'][filter_id],
                                   combined_gene_hyb_pos40x_y=d['combined_gene_hyb_pos40x_y'][filter_id],
                                   combined_gene_hyb_cellidall=d['combined_gene_hyb_cellidall'][filter_id],
                                   combined_gene_hyb_sliceidall=d['combined_gene_hyb_sliceidall'][filter_id])
    filtered_d=data_dict_organizer(filtered_d,'concat',combined_gene_hyb_id=[],combined_gene_hyb_fov=[],combined_gene_hyb_pos10x_x=[],combined_gene_hyb_pos10x_y=[],
                                   combined_gene_hyb_pos40x_x=[],combined_gene_hyb_pos40x_y=[],combined_gene_hyb_cellidall=[],combined_gene_hyb_sliceidall=[])

    
    cells=d['cell_list_all'].copy() # check if copy messed something
    genes=np.unique(d['combined_gene_hyb_id'])
    rol_id=d['combined_gene_hyb_id'].copy()
    rol_cell=d['combined_gene_hyb_cellidall'].copy()
    v=pd.crosstab(rol_cell, rol_id, rownames=['cell_index'], colnames=['genes'],dropna=False)
    v=v.reindex(index=cells, columns=genes, fill_value=0)
    exp_m=coo_matrix(v.to_numpy())
    processed_data={'all_data':d,
                    'filtered_data':filtered_d,
                    'expmat':exp_m,
                    'cells':cells,
                    'gene_id':genes,
                    'codebook_combined':codebook_comb}
    dump(processed_data,os.path.join(pth,'processed','processeddata.joblib'))


    rolonies={'id':filtered_d['combined_gene_hyb_id'],
              'pos10_x':filtered_d['combined_gene_hyb_pos10x_x'],
              'pos10_y':filtered_d['combined_gene_hyb_pos10x_y'],
              'pos40_x':filtered_d['combined_gene_hyb_pos40x_x'],
              'pos40_y':filtered_d['combined_gene_hyb_pos40x_y'],
              'slice':filtered_d['combined_gene_hyb_sliceidall'],
              'genes':codebook_comb,
              'fov':filtered_d['combined_gene_hyb_fov'],
              'fov_names':folders}
    neurons={'expmat':exp_m,
             'id':d['cell_list_all'],
             'pos10x_x':d['cell_pos_10x_allx'],
             'pos10x_y':d['cell_pos_10x_ally'],
             'pos40x_x':d['cell_pos_40x_allx'],
             'pos40x_y':d['cell_pos_40x_ally'],
             'slice':d['sliceidall_cell'],
             'genes':codebook_comb,
             'fov':d['fov_cell'],
             'fov_names':folders}
    dated_filename='alldata'+str(today)+'.joblib'
    dump({"rolonies":rolonies,"neurons":neurons},os.path.join(pth,'processed',dated_filename))
    print('ALL DATA IS ORGANIZED')
    return dated_filename

In [56]:
def data_dict_organizer(d,operation,**kwargs):
    """
    Helper function: Organizes dictionaries
    """
    if operation=='initialize':
        d.update(kwargs)
    elif operation=='append':
        for key in kwargs:
            d[key].append(kwargs[key])
    elif operation=='concat':
        for key in kwargs:
            d[key]=np.concatenate(d[key])
    return d

In [57]:
def merge_gene_hyb_dict(d,key1,key2,key3):
    """
    Helper function: Combines gene and hyb data into one dictionary
    """
    ar=[d[key1],d[key2]]
    d[key3]=np.concatenate(ar)
    return d

In [58]:
from scipy.spatial.distance import cdist
from scipy.sparse import coo_matrix
def filter_overlapping_cells(pth,fname='alldata06092025.joblib',rescale_factor=0.5,px=0.33,box_half_width_um=5,search_radius_um=200):
    """
    Postprocessing function:
    1. Removes overlapping neurons per tile
    2. Searching within a circle of search radius 200 um around per cell, looking for only cells from different fov and removing cells within a square of overlap width 10um
    3. Saves the final output filt_neurons
    """
    data=load(os.path.join(pth,'processed',fname))

    neurons=data['neurons']
    center_x=neurons['pos10x_x']
    center_y=neurons['pos10x_y']
    pr=px/rescale_factor
    overlap_half_width=np.round(box_half_width_um/pr)
    search_radius=np.round(search_radius_um/pr)
    xmin=center_x-overlap_half_width
    xmax=center_x+overlap_half_width
    ymin=center_y-overlap_half_width
    ymax=center_y+overlap_half_width
    c=np.column_stack((center_x,center_y))

    #max_dist=np.sqrt(2)*search_radius
    dist=cdist(c,c,'euclidean')
    dist_nearest=(dist<search_radius)
    [cell_id,nearest_neigh_id]=np.nonzero(dist_nearest)
    fov_neigh=neurons['fov'][nearest_neigh_id]
    fov_cell=neurons['fov'][cell_id]

    sel_cells_id=fov_cell!=fov_neigh
    distances_neigh=dist[cell_id[sel_cells_id],nearest_neigh_id[sel_cells_id]]
    search_cells_id=cell_id[sel_cells_id]
    search_neighbors_id=nearest_neigh_id[sel_cells_id]
    search_dist=distances_neigh
    id_overlap=((((xmin[search_cells_id]<xmin[search_neighbors_id])&(xmin[search_neighbors_id]<xmax[search_cells_id])) |
                 ((xmin[search_cells_id]<xmax[search_neighbors_id])&(xmax[search_neighbors_id]<xmax[search_cells_id]))) & 
                 (((ymin[search_cells_id]<ymin[search_neighbors_id])&(ymin[search_neighbors_id]<ymax[search_cells_id])) | 
                 ((ymin[search_cells_id]<ymax[search_neighbors_id])&(ymax[search_neighbors_id]<ymax[search_cells_id]))))
    overlap_cells_id=search_cells_id[id_overlap]
    overlap_neighbors_id=search_neighbors_id[id_overlap]
    overlap_distance=search_dist[id_overlap]
    # for i,idc in enumerate(overlap_cells_id):
    #     print(f"Cell {neurons['id'][idc]} in fov {neurons['fov_names'][neurons['fov'][idc]]} is matched to cell {neurons['id'][overlap_neighbors_id[i]]} in fov {neurons['fov_names'][neurons['fov'][overlap_neighbors_id[i]]]} with distance {overlap_distance[i]}")
    exp_mat=neurons['expmat'].todense() # I should do it in csr rather than dense--memory efficient
    total_exp_cell=np.asarray(np.sum(exp_mat,axis=1))
    [_,rev_idx]=np.unique(neurons['id'][overlap_cells_id],return_counts=False,return_inverse=True)
    df=pd.DataFrame({'cell':neurons['id'][overlap_cells_id],'neigh':neurons['id'][overlap_neighbors_id] ,'group':np.transpose(rev_idx),'neigh_exp':total_exp_cell[overlap_neighbors_id].flatten(),'cell_exp':total_exp_cell[overlap_cells_id].flatten()})
    idx_max=df.groupby('group')['neigh_exp'].idxmax()   
    # df=pd.DataFrame({'cell':neurons['id'][overlap_cells_id],'neigh':neurons['id'][overlap_neighbors_id] ,'neigh_exp':total_exp_cell[overlap_neighbors_id].flatten(),'cell_exp':total_exp_cell[overlap_cells_id].flatten()})
    # idx_max=df.groupby('cell')['neigh_exp'].idxmax()   
    df_ref=df.loc[idx_max]
    remove_cell=df_ref['neigh_exp']>df_ref['cell_exp']
    is_removed=np.zeros(center_x.shape)
    is_removed[overlap_cells_id[idx_max][remove_cell]]=1
    id_to_keep=is_removed==0
    filt_neurons={}
    expmat=coo_matrix(exp_mat[id_to_keep,:])
    filt_neurons['expmat']=expmat
    filt_neurons['id']=neurons['id'][id_to_keep]
    filt_neurons['pos10x_x']=neurons['pos10x_x'][id_to_keep]
    filt_neurons['pos10x_y']=neurons['pos10x_y'][id_to_keep]
    filt_neurons['pos40x_x']=neurons['pos40x_x'][id_to_keep]
    filt_neurons['pos40x_y']=neurons['pos40x_y'][id_to_keep]
    filt_neurons['slice']=neurons['slice'][id_to_keep]
    filt_neurons['genes']=neurons['genes']
    filt_neurons['fov']=neurons['fov'][id_to_keep]
    filt_neurons['fov_names']=neurons['fov_names']
    dump({"filt_neurons":filt_neurons,"removecells_all":id_to_keep},os.path.join(pth,'processed','filt_neurons.joblib'))
    print('OVERLAPPING CELLS REMOVED--PROCESSING FINISHED')

In [59]:
import subprocess,yaml
def run_diff_env_scripts(name,config_name, **params):
    cfg=yaml.safe_load(open(config_name))
    spec=cfg["tasks"][name]
    argv=[item.format(**params) for item in spec["argv"]]
    env=spec["env"].split(":",1)[1]
    cmd=["conda", "run", "-n", env, "python", *argv]
    subprocess.run(cmd, check=True)
    

In [60]:
def preprocessing(pth,config_pth,basedir,chshift_filename_geneseq,chshift_filename_bcseq,chshift_filename_hyb,chprofile_filename_geneseq,chprofile_filename_bcseq,chprofile_filename_hyb,
                 config_file='cfg.yaml',is_affine=0,num_initial_c=5,num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2,block_size=128,radius=31,local=1,
                 cyclename_geneseq='n2vgene',cyclename_bcseq='regn2vbc',cycle_name_hyb='n2vhyb',strength=0.6, m=0,bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5,bcfname='n2vbcseq',
                 num_c_hyb=5,cyclename_rgb='n2vhyb01'):

    process_geneseq_cycle(pth,config_pth,basedir,chshift_filename_geneseq,chprofile_filename_geneseq,config_file=config_file,is_affine=is_affine,num_initial_c=num_initial_c,num_later_c=num_later_c,
                          num_c=num_c,subsample_rate=subsample_rate,do_coarse=do_coarse,resize_factor=resize_factor, block_size=block_size,radius=radius,local=local,cyclename=cyclename_geneseq,
                          strength=strength,m=m)

    process_bc_cycle(pth,config_pth,basedir,chshift_filename_bcseq,chprofile_filename_bcseq,config_file=config_file,is_affine=is_affine,num_initial_c=num_initial_c,num_later_c=num_later_c,
                     num_c=num_c,subsample_rate=subsample_rate,do_coarse=do_coarse,resize_factor=resize_factor, block_size=block_size,radius=radius,local=local,cyclename=cyclename_bcseq, 
                     bcidx=bcidx,geneidx=geneidx,ch_bc=ch_bc,ch_bc_init=ch_bc_init,bcfname=bcfname,genefname=cyclename_geneseq,strength=strength,m=m)
    process_hyb_cycle(pth, config_pth,basedir,chshift_filename_hyb,chprofile_filename_hyb,config_file=config_file,cycle_name=cycle_name_hyb,num_c=num_c_hyb,is_affine=is_affine,
                      cyclename_rgb=cyclename_rgb,strength=strength)

In [61]:
def stitch_and_segment(pth,pth_fiji,fname='n2vgeneseq01',stitching_channel=3,overlap=0.23,numcores=8,is_headless=True,grid_direction='UR',px=0.33,num_c=4,
                       intensity_scaling=3,transform_rescale_factor=0.5,display_additional_rescale=0.2,
                       model_name='cyto3',outname='cell_mask_cyto3.tif',nuc_ch=5,num_chyb=5,num_cgene=4,
                       tilesize=3200,diameter=40,config_file='cfg.yaml',dilation_radius=3, stitch_tool='MIST'):
    if stitch_tool=='MIST':
        sx,sy=stitch_mist_allpos(pth,pth_fiji,fname,stitching_channel,overlap,numcores,is_headless,grid_direction,px,transform_rescale_factor,num_c)
        produce_tform_and_stitched_images(pth,sx,sy,intensity_scaling,tilesize,transform_rescale_factor,display_additional_rescale)
    else:
        # I DON'T HAVE ASHLAR STICTHING SCRIPT HERE
        sx,sy=merge_ashlar_results(pth,transform_rescale_factor,num_c)
        produce_tform_and_stitched_images(pth,sx,sy,intensity_scaling,tilesize,transform_rescale_factor,display_additional_rescale)
        
    
    prepare_cellpose_input(pth,nuc_ch,num_chyb,num_cgene,tilesize)
    run_diff_env_scripts("cellpose_segmentation",config_file,pth=pth,diameter=diameter,outname=outname,model_name=model_name)
    import_cellpose_all_tiles(pth,dilation_radius,outname)

In [62]:
def basecalls(pth,config_pth,num_channels=4,codebook_name='codebookM1all.mat',codebook_name2='codebook.joblib',fdrthresh=0.05,trim=160,cropf=0.4,noisefloor_ini=0.01,noisefloor_final=0.05,
              prev_codebook_len=0,codebook_opt_name='codebook.mat',no_deconv=1,codebook_hyb_name='codebookhyb.mat',cyclename_hyb='alignedn2vhyb',num_c_hyb=4,thresh=[50,60,40,30],
              prominence=[50,10,40,10],all_genes_ch=2,is_optseq=0):
    
    bardensr_call(pth,config_pth,num_channels,codebook_name,fdrthresh,trim,cropf,noisefloor_ini,noisefloor_final)
    import_bardensr_results(pth,'bardensrresult.csv',prev_codebook_len,codebook_name2,is_optseq,codebook_opt_name)
    basecall_hyb_all_tiles(pth,cyclename_hyb,config_pth,codebook_hyb_name,thresh,prominence,num_c_hyb,all_genes_ch,no_deconv)

In [64]:
def data_aggregation_and_filtering(pth,config_pth,is_optseq=0,hyb_codebook_name='codebookhyb.mat',starting_slice_idx=1,starting_fov_idx=1,dummy_cell_num=10000,tilesize=3200,fraction_border=0.07,
                                   rescale_factor=0.5,px=0.33,box_half_width_um=5,search_radius_um=200,is_bc=0):
    aggregate_cell_ids_gene_hyb(pth,is_bc)
    transform_downsized_coordinates(pth,is_bc)
    fname=organize_processed_data(pth,config_pth,is_optseq,hyb_codebook_name,starting_slice_idx,starting_fov_idx,dummy_cell_num,tilesize,fraction_border)
    filter_overlapping_cells(pth,fname,rescale_factor,px,box_half_width_um,search_radius_um)

In [65]:
def check_geneseq_cycle_alignment(pth,folder,n_cycles=7):
    I=[]

    fg, ax=plt.subplots(int(np.round(n_cycles/4)),4)
    ax=ax.reshape(-1)
    for i in range(n_cycles):
        I.append(np.sum(tfl.imread(os.path.join(pth,'processed',folder,'aligned','alignedn2vgeneseq'+str("%0.2d"%(i+1))+'.tif'),key=range(0,4,1)),axis=0))
        I1=I[0]
        I2=I[i]
        ax[i].imshow(make_overlay(I1,I2)[1450:1550,1350:1450,:])
    fg.set_figheight(10)
    fg.set_figwidth(20)
    fg.suptitle('Geneseq to first cycle overlaps', fontsize=16)
    plt.tight_layout()



In [66]:
def check_bcseq_cycle_alignment(pth,folder,n_cycles):
    I=[]

    fg, ax=plt.subplots(4,int(np.round(n_cycles/4)))
    ax=ax.reshape(-1)

    for i in range(n_cycles):
        I.append(np.sum(tfl.imread(os.path.join(pth,'processed',folder,'aligned','alignedregn2vbcseq'+str("%0.2d"%(i+1))+'.tif'),key=range(0,4,1)),axis=0))
        I1=I[0]
        I2=I[i]
        ax[i].imshow(make_overlay(I1,I2)[1450:1550,1350:1450,:])
    fg.set_figheight(20)
    fg.set_figwidth(20)
    fg.suptitle('Bcseq to first cycle overlaps', fontsize=16)

    plt.tight_layout()


In [67]:
def make_overlay(I1,I2):
    I_rgb=np.zeros((3,I1.shape[0],I1.shape[1]))
    I_rgb[0,:,:]=np.double(np.divide(I1,np.max(I1,axis=None)))*5
    I_rgb[1,:,:]=np.double(np.divide(I2,np.max(I2,axis=None)))*5
    Irgb=np.uint8(np.clip(I_rgb*255,0,255))
    return np.transpose(Irgb,(1,2,0))

In [68]:
def check_hyb_basecall(pth,config_pth,fov=9,codebook_hyb_name='codebookhyb.mat'):
    codebook_hyb_path=os.path.join(config_pth,codebook_hyb_name)
    codebook_hyb=scipy.io.loadmat(codebook_hyb_path)['codebookhyb']
    L=load(os.path.join(pth,'processed','genehyb.joblib'))
    fg,ax=plt.subplots()
    for i in range(len(L['lroi_x'][fov])):
        ax.scatter(L['lroi_y'][fov][i], L['lroi_x'][fov][i],s=3)#,c=col[i])
        ax.invert_yaxis()
    plt.legend([codebook_hyb[0][0],codebook_hyb[1][0],codebook_hyb[2][0]])
    fg.suptitle('Hyb basecall', fontsize=16)

    plt.tight_layout()

In [69]:
def check_stitched_output(pth,if_bcseq=1,if_hyb=1,if_geneseq=1,if_mist=1):
    fg, ax=plt.subplots(1,3)
    fg.suptitle('Stitched', fontsize=16)
    reg_folder='checkregistration'
    
    stitch_output=os.path.join(pth,'processed',reg_folder)
    if if_geneseq:
        geneseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vgeneseq01.tif'))
        ax[0].imshow(geneseq_pos)
    if if_bcseq:
        bcseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedregn2vbcseq01.tif'))
        ax[1].imshow(bcseq_pos)
    if if_hyb:
        hybseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vhyb01.tif'))
        ax[2].imshow(hybseq_pos)
    fg.set_figheight(5)
    fg.set_figwidth(10)
    fg.tight_layout()
    

In [70]:
def check_final_overlap_removal(pth,data_file):
    data2=load(os.path.join(pth,'processed','filt_neurons.joblib'))
    data=load(os.path.join(pth,'processed',data_file))
    neurons=data['neurons']
    filt_neurons=data2['filt_neurons']
    id_to_keep=data2['removecells_all']
    fg, ax=plt.subplots(1,2)
    fg.suptitle('Before and after overlap filter', fontsize=16)
    fg.set_figheight(5)
    fg.set_figwidth(10)
    
    exp_mat=neurons['expmat'].todense() # I should do it in csr rather than dense--memory efficient
    total_exp_cell=np.asarray(np.sum(exp_mat,axis=1))
    ax[0].scatter(neurons['pos10x_x'],neurons['pos10x_y'],1,total_exp_cell,cmap=plt.cm.viridis_r)
    ax[1].scatter(filt_neurons['pos10x_x'],filt_neurons['pos10x_y'],1,total_exp_cell[id_to_keep],cmap=plt.cm.viridis_r)
    fg.tight_layout()


In [71]:
def check_cellpose_output(pth,folder):
    I=np.sum(tfl.imread(os.path.join(pth,'processed',folder,'aligned','cell_inp2.tif'),key=range(0,2)),axis=0)
    mask=tfl.imread(os.path.join(pth,'processed',folder,'aligned','cell_mask_cyto3.tif'))
    mask_dil=expand_labels(mask)
    fg, ax=plt.subplots(1,2)
    fg.suptitle('Cell labels', fontsize=16)
    ax[0].imshow(I)
    ax[1].imshow(mask_dil)
    plt.tight_layout()
    fg.set_figheight(10)
    fg.set_figwidth(20)
    


In [72]:
def check_gene_expression(pth,gene_idx):
    fig = plt.figure()
    data2=load(os.path.join(pth,'processed','filt_neurons.joblib'))
    filt_neurons=data2['filt_neurons']
    genes=np.unique(filt_neurons['id'])
    exp_m=filt_neurons['expmat']
    idg=exp_m.col==gene_idx
    plt.scatter(filt_neurons['pos10x_x'][exp_m.row[idg]],filt_neurons['pos10x_y'][exp_m.row[idg]],2,np.log(1+exp_m.data[idg]),cmap='cool')
    #plt.savefig(os.path.join(pth,'processed','png',str(codebook_comb[gene_idx][0])+'.png'),dpi=300)
    fig.suptitle(f"{filt_neurons['genes'][gene_idx][0]} expression", fontsize=16)

    plt.show()

In [73]:
def check_cycle_registration(pth,thresh,modality,n_cycles=7):
    [folders,pos,_,_]=get_folders(pth)
    N=len(folders)
    v=np.zeros([N,n_cycles])
    cont=np.zeros([N,1])
    #q=random.randint(1, len(folders))
    t0=time.time()
    for q in range(N):
        print(f"folder {folders[q]}")
        for i in range(n_cycles):
            I2=np.sum(tfl.imread(os.path.join(pth,'processed',folders[q],'aligned','alignedfixedn2vgeneseq'+str("%0.2d"%(i+1))+'.tif'),key=range(0,4,1)),axis=0)
            if i==0:
                I1=I2
            v[q,i]=nmi(I1,I2)
        cont[q,0]=np.var(I1)

    variance_fov=np.zeros([v.shape[0],1])
    eps=0
    for r in range(v.shape[0]):
        vr=((v[r,1:]-np.min(v[r,1:]))/(eps+np.max(v[r,1:])-np.min(v[r,1:])))
        variance_fov[r]=np.max(vr)/(eps+np.median(vr))
    
    idx=np.abs(variance_fov)>thresh
    idx=idx.reshape(-1)
    folders=np.array(folders)
    sel_folders=folders[idx]
    sel_folders_score=variance_fov[idx]
    sel_folders_cont=cont[idx]
    tn=time.time()
    dump({'name':sel_folders,'score':sel_folders_score,'content':sel_folders_cont},os.path.join(pth,'processed',modality+'cycle_registration_metric.joblib'))


    print(f"Total time {tn-t0} seconds\n")

In [74]:
def stitch_images_RGB_cycles(pth,intensity_scaling=3,tilesize=3200,transform_rescale_factor=0.5,display_additional_rescale=1):
    [folders,pos,x,y]=get_folders(pth)
    unique_pos=nsort(np.unique(pos))
    check_reg_folder=os.path.join(pth,'processed','checkregistration_rgb_cycles')
    os.makedirs(check_reg_folder,exist_ok=True)
    folder_names=np.array(folders)
    filenames=sorted(glob.glob(os.path.join(pth,'processed','gene_cycle_pngs',"*.png")))
    T=load(os.path.join(pth,'tforms_rescaled'+str(transform_rescale_factor).replace('.','p')+'.joblib'))
    sx=[]
    sy=[]
    for n_pos in unique_pos:
        pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
        for ids in pos_id:
            tilename=folder_names[ids]+'.tif'
            sx.append(T[n_pos][tilename]["ref_pos"][0])
            sy.append(T[n_pos][tilename]["ref_pos"][1])
   

    for n_pos in unique_pos:
        I=np.zeros([int(np.max(sy)+tilesize*transform_rescale_factor),int(np.max(sx)+tilesize*transform_rescale_factor),3])
        pos_id=np.array([i for i,name in enumerate(pos) if name==n_pos])
        for ids in pos_id:
            tform=load(os.path.join(pth,'processed',folder_names[ids],'global_tform_'+str(transform_rescale_factor).replace('.','p')+'.joblib'))
            tile=tfl.imread(filenames[ids])
            It=skimage.transform.warp(np.squeeze(tile),tform.inverse,preserve_range=True,output_shape=(I.shape[0],I.shape[1]))
            I=np.maximum(I,It)
        Irgb=np.uint8(np.clip(I*intensity_scaling,0,255))
        Irgb_rescaled = np.uint8(np.clip(rescale(Irgb, display_additional_rescale, channel_axis=-1, preserve_range=True, anti_aliasing=True),0,255))
        tfl.imwrite(os.path.join(check_reg_folder,n_pos+'.png'),Irgb_rescaled,photometric='rgb')

In [75]:
# NEED TO ADD SOME CHECKPOINTS SO THAT WHATEVER IS DONE IS NOT REPEATED
# NEED TO ADD CHECKPOINTS TO SEE IF DIFFERENT MODALITIES EXIST IN THIS EXPERIMENT

pth='/mnt/d/allen_data/test_dataset_bc_726126/'
pth_fiji = r'/home/nrg/Fiji.app' 
config_pth='/mnt/c/Users/goswami/Documents/MATLAB/barseq-matlab-allen-main (1)/barseq-matlab-allen-main/config/'
basedir = '/home/nrg/n2vmodels/'


chshift_filename_geneseq='chshift20x-20220218.mat'
chprofile_filename_geneseq='chprofile20x-50-30-20-40-20220218.mat'

chshift_filename_bcseq='chshift20x-20220218.mat'
chprofile_filename_bcseq='chprofile20x-50-30-20-40-20220218.mat'

chshift_filename_hyb='chshift20x-hyb-20220218.mat'
chprofile_filename_hyb='chprofilehyb_allen.mat'



preprocessing(pth,config_pth,basedir,chshift_filename_geneseq,chshift_filename_bcseq,chshift_filename_hyb,chprofile_filename_geneseq,chprofile_filename_bcseq,chprofile_filename_hyb,
                 config_file='cfg.yaml',is_affine=0,num_initial_c=5,num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2,block_size=128,radius=31,local=1,
                 cyclename_geneseq='n2vgene',cyclename_bcseq='regn2vbc',cycle_name_hyb='n2vhyb',strength=0.6, m=0,bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5,bcfname='n2vbcseq',
                 num_c_hyb=5,cyclename_rgb='n2vhyb01')
stitch_and_segment(pth,pth_fiji,fname='n2vgeneseq01',stitching_channel=3,overlap=0.23,numcores=8,is_headless=True,grid_direction='UR',px=0.33,num_c=4,
                       intensity_scaling=3,transform_rescale_factor=0.5,display_additional_rescale=0.2,
                       model_name='cyto3',outname='cell_mask_cyto3.tif',nuc_ch=5,num_chyb=5,num_cgene=4,
                       tilesize=3200,diameter=40,config_file='cfg.yaml',dilation_radius=3)
basecalls(pth,config_pth,num_channels=4,codebook_name='codebookM1all.mat',codebook_name2='codebook.joblib',fdrthresh=0.05,trim=160,cropf=0.4,noisefloor_ini=0.01,noisefloor_final=0.05,
              prev_codebook_len=0,codebook_opt_name='codebook.mat',no_deconv=1,codebook_hyb_name='codebookhyb.mat',cyclename_hyb='alignedn2vhyb',num_c_hyb=4,thresh=[50,60,40,30],
              prominence=[50,10,40,10],all_genes_ch=2,is_optseq=0)
data_aggregation_and_filtering(pth,config_pth,is_optseq=0,hyb_codebook_name='codebookhyb.mat',starting_slice_idx=1,starting_fov_idx=1,dummy_cell_num=10000,tilesize=3200,fraction_border=0.07,
                                   rescale_factor=0.5,px=0.33,box_half_width_um=5,search_radius_um=200,is_bc=1)

Copying files in the correct folders...
Organizing folder /mnt/d/allen_data/test_dataset_bc_726126/geneseq01

Copied 0


KeyboardInterrupt: 

In [ ]:
# RUN SOME TESTS
data_file='alldata19092025.joblib'
gene_idx=111
[folders,pos,_,_]=get_folders(pth)
q=9
folder=folders[q]
check_geneseq_cycle_alignment(pth,folder,7)
check_bcseq_cycle_alignment(pth,folder,15)
check_hyb_basecall(pth,config_pth,fov=q,codebook_hyb_name='codebookhyb.mat')
check_stitched_output(pth,if_bcseq=1,if_hyb=1,if_geneseq=1,if_mist=1)
check_cellpose_output(pth,folder)
check_final_overlap_removal(pth,data_file)
check_gene_expression(pth,gene_idx)
stitch_images_RGB_cycles(pth,intensity_scaling=3,tilesize=3200,transform_rescale_factor=0.5,display_additional_rescale=1)

In [ ]:

pth='/mnt/d/allen_data/test_dataset_bc_726126/processed/MAX_Pos1_000_003/original/'

config_pth='/mnt/c/Users/goswami/Documents/MATLAB/barseq-matlab-allen-main (1)/barseq-matlab-allen-main/config/'
cyclename='n2vgene'
is_affine=0
num_initial_c=5 # initial channels in first round of sequencing to read
num_later_c=4 # subsequent cycle channels to read
num_c=4 # channels to perform bleedthrough-correction and channel alignment, needs more elegant solution--ng
subsample_rate=4
do_coarse=0
resize_factor=2
block_size=128
chshift_filename='chshift20x-20220218.mat'
chprofile_filename='chprofile20x-50-30-20-40-20220218.mat'
radius=31
local=1

In [ ]:
I=tfl.imread(os.path.join(pth,'n2vgeneseq02.tif'),key=range(0,4,1))
Ifilt=back_sub_opencv_open(I,31,pth,'bcksb.tif',4,0)

fg, ax=plt.subplots(1,2)
ax[0].imshow(I[3,:,:],cmap='jet',vmin=0,vmax=150)
ax[1].imshow(Ifilt[3,:,:],cmap='jet',vmin=0,vmax=150)
plt.tight_layout()


In [ ]:
Ialigned=channel_alignment(Ifilt,chshift_filename,config_pth,pth,'aligned.tif',4,0,0)
fg, ax=plt.subplots(1,2)
ax[0].imshow(np.sum(Ifilt[:,362:372,450:462],axis=0),cmap='jet',vmin=0,vmax=150)
ax[1].imshow(np.sum(Ialigned[:,362:372,450:462],axis=0),cmap='jet',vmin=0,vmax=150)
plt.tight_layout()

In [ ]:
Icorr=bleedthrough_linear(Ialigned,4,config_pth,chprofile_filename,pth,'corr.tif',0)
fg, ax=plt.subplots(2,4)
for i in range(4):
    ax[0,i].imshow(Ialigned[i,362:372,450:462],cmap='jet',vmin=0,vmax=100)
    ax[1,i].imshow(Icorr[i,362:372,450:462],cmap='jet',vmin=0,vmax=100)
plt.tight_layout()

In [ ]:
#CHECK GENESEQ CYCLE ALIGNEMENT
pth='/mnt/d/allen_data/test_dataset_bc_726126/processed/MAX_Pos1_000_003/aligned/'
I1=np.sum(tfl.imread(os.path.join(pth,'alignedn2vgeneseq01.tif'),key=range(0,4,1)),axis=0)
I2=np.sum(tfl.imread(os.path.join(pth,'alignedn2vgeneseq07.tif'),key=range(0,4,1)),axis=0)
I_rgb=np.zeros((3,I1.shape[0],I1.shape[1]))
I_rgb[0,:,:]=np.double(np.divide(I1,np.max(I1,axis=None)))*10
I_rgb[1,:,:]=np.double(np.divide(I2,np.max(I2,axis=None)))*10


Irgb=np.uint8(np.clip(I_rgb*255,0,255))

pthb='/mnt/d/allen_data/test_dataset_bc_726126/726126_test_dataset_processed/processed/MAX_Pos1_000_003/aligned/MAX_Pos1_000_003/aligned'
I1b=np.sum(tfl.imread(os.path.join(pthb,'alignedfixedn2vgeneseq01.tif'),key=range(0,4,1)),axis=0)
I2b=np.sum(tfl.imread(os.path.join(pthb,'alignedfixedn2vgeneseq07.tif'),key=range(0,4,1)),axis=0)
I_rgbb=np.zeros((3,I1b.shape[0],I1b.shape[1]))
I_rgbb[0,:,:]=np.double(np.divide(I1b,np.max(I1b,axis=None)))*10
I_rgbb[1,:,:]=np.double(np.divide(I2b,np.max(I2b,axis=None)))*10


Irgbb=np.uint8(np.clip(I_rgbb*255,0,255))


fg, ax=plt.subplots(1,2)
ax[0].imshow(np.transpose(Irgb[:,1450:1550,1350:1450],(1,2,0)),vmin=0,vmax=1)
ax[1].imshow(np.transpose(Irgbb[:,1450:1550,1350:1450],(1,2,0)),vmin=0,vmax=1)
plt.tight_layout()



In [ ]:
#CHECK BCSEQ CYCLE ALIGNEMENT
pth='/mnt/d/allen_data/test_dataset_bc_726126/processed/MAX_Pos1_000_003/aligned/'
I1=np.sum(tfl.imread(os.path.join(pth,'alignedregn2vbcseq01.tif'),key=range(0,4,1)),axis=0)
I2=np.sum(tfl.imread(os.path.join(pth,'alignedregn2vbcseq05.tif'),key=range(0,4,1)),axis=0)
I_rgb=np.zeros((3,I1.shape[0],I1.shape[1]))
I_rgb[0,:,:]=np.double(np.divide(I1,np.max(I1,axis=None)))*10
I_rgb[1,:,:]=np.double(np.divide(I2,np.max(I2,axis=None)))*10


Irgb=np.uint8(np.clip(I_rgb*255,0,255))

pthb='/mnt/d/allen_data/test_dataset_bc_726126/726126_test_dataset_processed/processed/MAX_Pos1_000_003/aligned/MAX_Pos1_000_003/aligned'
I1b=np.sum(tfl.imread(os.path.join(pthb,'alignedfixedregn2vbcseq01.tif'),key=range(0,4,1)),axis=0)
I2b=np.sum(tfl.imread(os.path.join(pthb,'alignedfixedregn2vbcseq05.tif'),key=range(0,4,1)),axis=0)
I_rgbb=np.zeros((3,I1b.shape[0],I1b.shape[1]))
I_rgbb[0,:,:]=np.double(np.divide(I1b,np.max(I1b,axis=None)))*10
I_rgbb[1,:,:]=np.double(np.divide(I2b,np.max(I2b,axis=None)))*10


Irgbb=np.uint8(np.clip(I_rgbb*255,0,255))


fg, ax=plt.subplots(1,2)
ax[0].imshow(np.transpose(Irgb[:,1450:1550,1350:1450],(1,2,0)),vmin=0,vmax=1)
ax[1].imshow(np.transpose(Irgbb[:,1450:1550,1350:1450],(1,2,0)),vmin=0,vmax=1)
# ax[0].imshow(np.transpose(Irgb[:,:.:],(1,2,0)),vmin=0,vmax=1)
# ax[1].imshow(np.transpose(Irgbb[:,:.:],(1,2,0)),vmin=0,vmax=1)
plt.tight_layout()

In [ ]:
#HYB FINAL LAYER-----------------develop
# pthb='/mnt/d/allen_data/test_dataset_bc_726126/'

# cycle_name='n2vhyb'
# config_pth='/mnt/c/Users/goswami/Documents/MATLAB/barseq-matlab-allen-main (1)/barseq-matlab-allen-main/config/'
# chshift_hyb_filename='chshift20x-hyb-20220218.mat'
# chprofile_hyb_filename='chprofilehyb_allen.mat'
# num_c=5
# is_affine=0
# cyclename='n2vhyb01'
# strength=0.6


In [ ]:
# PROCESS BC IMAGES
pthb='/mnt/d/allen_data/test_dataset_bc_726126'

process_bc_cycle(pthb,config_pth,chshift_filename,chprofile_filename,is_affine=0,num_initial_c=5,num_later_c=4,num_c=4,subsample_rate=4,do_coarse=0,resize_factor=2,block_size=128,radius=31,local=1,cyclename='regn2vbc',bcidx=4,geneidx=4,ch_bc=4,ch_bc_init=5,bcfname='n2vbcseq',genefname='n2vgene',strength=0.6,m=0)



In [ ]:
# BARDENSR CALL
t0=time.time()
pth='/mnt/d/allen_data/processed/'
bardensr_call(pth,config_pth,num_channels=4,codebook_name='codebookM1all.mat',fdrthresh=0.05,trim=160,cropf=0.4,noisefloor_ini=0.01,noisefloor_final=0.05)
tn=time.time()
print(f'Total time for {len(folders)} is {tn-t0} secs')

In [ ]:
# IMPORT BARDENSR RESULTS ACROSS WHOLE DATASET
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
fname='bardensrresult.csv'
import_bardensr_results(pth,fname='bardensrresult.csv',prev_codebook_len=0,codebook_name='codebook.mat',id_optseq=0,codebook_opt_name='codebook.mat')

In [ ]:
# HYB BASE CALL------------------------------written in a way that finds max for all spots across channels
from skimage.measure import label
from skimage import color
from skimage.morphology import extrema
from skimage.util import img_as_float
from skimage.exposure import rescale_intensity
relaxed=1;
no_deconv=1;
filter_overlap=0;
codebook_hyb_name='codebookhyb.mat'
config_pth='/mnt/c/Users/goswami/Documents/MATLAB/barseq-matlab-allen-main (1)/barseq-matlab-allen-main/config/'
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
cyclename='alignedn2vhyb'
num_c=4
thresh=[50,60,40,30]
prominence=[50,10,40,10]
all_genes_ch=2
basecall_hyb_all_tiles(pth,cyclename,config_pth,codebook_hyb_name,thresh,prominence,num_c,all_genes_ch,no_deconv)


In [ ]:
# CHECK HYB BASE CALL
fov=9
codebook_hyb_name='codebookhyb.mat'
config_pth='/mnt/c/Users/goswami/Documents/MATLAB/barseq-matlab-allen-main (1)/barseq-matlab-allen-main/config/'
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
codebook_hyb_path=os.path.join(config_pth,codebook_hyb_name)
codebook_hyb=scipy.io.loadmat(codebook_hyb_path)['codebookhyb']
print(codebook_hyb[0])
fg,ax=plt.subplots()
for i in range(len(L['lroi_x'][fov])):
    ax.scatter(L['lroi_y'][fov][i], L['lroi_x'][fov][i],s=3)#,c=col[i])
    ax.invert_yaxis()
plt.legend([codebook_hyb[0][0],codebook_hyb[1][0],codebook_hyb[2][0]])
plt.tight_layout()

In [ ]:
# STITCH--MIST
sx,sy=stitch_mist_allpos(pth,pth_fiji,fname='n2vgeneseq01',stitching_channel=3,overlap=0.23,numcores=8,is_headless=True,grid_direction='UR',px=0.33,transform_rescale_factor=0.5,num_c=4)
produce_tform_and_stitched_images(pth,sx,sy,intensity_scaling=3,tilesize=3200,transform_rescale_factor=0.5,display_additional_rescale=0.2)

In [ ]:
# STITCH--ASHLAR---main stitching routine for ASHLAR is not implemented here, it is with John
sx,sy=merge_ashlar_results(pth,transform_rescale_factor=0.5,num_c=4)
produce_tform_and_stitched_images(pth,sx,sy,intensity_scaling=3,tilesize=3200,transform_rescale_factor=0.5,display_additional_rescale=0.2)

In [ ]:
# CHECK STITCHED OUTPUT-MIST
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
stitch_output=os.path.join(pth,'processed','checkregistration_mist')
geneseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vgeneseq01.tif'))
bcseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedregn2vbcseq01.tif'))
hybseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vhyb01.tif'))
fg, ax=plt.subplots(1,3)
ax[0].imshow(geneseq_pos)
ax[1].imshow(bcseq_pos)
ax[2].imshow(hybseq_pos)
fg.set_figheight(10)
fg.set_figwidth(10)
plt.tight_layout()

In [ ]:
# CHECK STITCHED OUTPUT-ASHLAR
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
stitch_output=os.path.join(pth,'processed','checkregistration')
geneseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vgeneseq01.tif'))
bcseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedregn2vbcseq01.tif'))
hybseq_pos=tfl.imread(os.path.join(stitch_output,'Pos1_RGBalignedn2vhyb01.tif'))
fg, ax=plt.subplots(1,3)
ax[0].imshow(geneseq_pos)
ax[1].imshow(bcseq_pos)
ax[2].imshow(hybseq_pos)
fg.set_figheight(10)
fg.set_figwidth(10)
plt.tight_layout()

In [ ]:
#CELLPOSE
outname='cell_mask_cyto3.tif'
model_name='cyto3'
prepare_cellpose_input(pth,nuc_ch=5,num_chyb=5,num_cgene=4,tilesize=3200)
run_diff_env_scripts("cellpose_segmentation",'cfg.yaml', pth='/mnt/d/allen_data/test_dataset_bc_726126/', diameter=40, outname=outname,model_name=model_name)
import_cellpose_all_tiles(pth,dilation_radius=3,fname=outname)


In [ ]:
# DATA AGGREGATION
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
aggregate_cell_ids_gene_hyb(pth,is_bc)
transform_downsized_coordinates(pth,is_bc))

In [ ]:
# DATA ORGANIZATION
organize_processed_data(pth,config_pth,is_optseq=0,hyb_codebook_name='codebookhyb.mat',starting_slice_idx=1,starting_fov_idx=1,dummy_cell_num=10000,tilesize=3200,
                           fraction_border=0.07)

In [ ]:
# FILTER OVER;APPING NEURONS
pth='/mnt/d/allen_data/test_dataset_bc_726126/'
filter_overlapping_cells(pth,fname='alldata06092025.joblib',rescale_factor=0.5,px=0.33,box_half_width_um=5,search_radius_um=200)


In [ ]:
# DISPLAY OVERLAP REMOVAL RESULTS
data2=load(os.path.join(pth,'processed','filt_neurons.joblib'))
data=load(os.path.join(pth,'processed','alldata06092025.joblib'))
neurons=data['neurons']
filt_neurons=data2['filt_neurons']
id_to_keep=data2['removecells_all']
fg, ax=plt.subplots(1,2)
fg.set_figheight(5)
fg.set_figwidth(10)
plt.tight_layout()
exp_mat=neurons['expmat'].todense() # I should do it in csr rather than dense--memory efficient
total_exp_cell=np.asarray(np.sum(exp_mat,axis=1))
id_to_keep
ax[0].scatter(neurons['pos10x_x'],neurons['pos10x_y'],1,total_exp_cell,cmap=plt.cm.viridis_r)
ax[1].scatter(filt_neurons['pos10x_x'],filt_neurons['pos10x_y'],1,total_exp_cell[id_to_keep],cmap=plt.cm.viridis_r)
print(np.sum(id_to_keep))